In [1]:
####### final version: DEMTECT subtask REVERSE
# Setup for the notebook
#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import nengo
import nengo_spa as spa
from nengo.networks import CircularConvolution
from nengo_spa import Vocabulary
from nengo_spa import SemanticPointer
from nengo_spa import State
from nengo_spa import AssociativeMemory
from nengo_spa import ThresholdingAssocMem
from nengo_spa import WTAAssocMem

# vocabulary 
D = 64 # 512
n_neurons = 50 # number of neurons per dimension

# buffers 
dimensions = D
subdims = 1
n_neurons = 50            # 50 # number of neurons per dimension

# ------------------------------------------------------------------------------------------------------------------------------
# functions: 

# --------------------------------------------------------------------------------------------------------------------------
# Pick a dimension for our vectors. Higher dimensions generally provide a more accurate encoding,
# but also require more neural resources.


# create mappings from key-vectors: elemets in key-vectors need to be in correct order
def create_mapping (input_keys, output_keys):
    mapping_in_out = {}
    
    for i in range(0,len(input_keys)):
        mapping_in_out[input_keys[i]] = output_keys[i]

    return mapping_in_out

# always produce id_s-pointers as well:  
def encode_semantic_network(semantic_network, relation_types, D):
    """ Encode a semantic network in vectors. """
    
    #print (semantic_network)

    # Notice we don't have to do any funky topological sorting this time around.
    rt_vocab = Vocabulary(D, strict=False)
    id_vocab = Vocabulary(D, strict=False)
    sp_vocab = Vocabulary(D, strict=False)

    from nengo_spa.vector_generation import UnitLengthVectors
    pointer_gen = UnitLengthVectors(D)

    for rt in relation_types:
        rt_vocab.parse(rt)
    
    for concept in semantic_network:
        id_vocab.parse(concept)

    for concept in semantic_network:
        semantic_pointer = SemanticPointer(next(pointer_gen))

        for relation in semantic_network[concept]:
            rt = rt_vocab.parse(relation[0])
            id = id_vocab.parse(relation[1])
        
            p = SemanticPointer(rt.v) * SemanticPointer(id.v)
            semantic_pointer += p

        sp_vocab.add(concept, semantic_pointer.normalized())
        
    return sp_vocab, rt_vocab, id_vocab

# ------------------------------------------------------------------------------------------------------------------------------
# Semantic Network: Python Dictionary:
# semantic relations at concept level:
assocWith = 'AssocWith' # ... belongs to a superordinate concept (gehbört zur Gruppe der): ...
buyIn = 'BuyIn' # ... Supermarkt, Bäckerei, hardware store, ....


# phonological network: 
# phonological relations at syllable level: 
inclPhon = 'InclPhon'   # includes phonologically / phonetically sub-constituents (syllable parts, single phones) .... 

# define relation types: 
rel_type_concept_keys = [assocWith, buyIn] 
rel_type_phono_keys = [inclPhon]  


In [2]:
########################################################################################################################################
# Begin: mental lexicon and key arrays -> Catharina Stille
# the (semantic) network: 

phonos_deep = {
        # weiteres inclPhon wäre auch hier möglich! -> weitere deep-Level  
        # phonologische Ähnlickleit wird nur für die betonten Silben aufgebaut!!!
        'Pn_St_a': [],
        'Pn_St_aa': [],
        'Pn_St_aI': [],
        'Pn_St_aU': [],
        'Pn_St_ba': [],
        'Pn_St_baa': [],
        'Pn_St_bE': [],
        'Pn_St_bi': [],
        'Pn_St_bI': [],
        'Pn_St_bl': [],
        'Pn_St_br': [],
        'Pn_St_bU': [],
        'Pn_St_buu': [],
        'Pn_St_bY': [],
        'Pn_St_byy': [],
        'Pn_St_da': [],
        'Pn_St_dE': [],
        'Pn_St_dee': [],
        'Pn_St_dii': [],
        'Pn_St_dr': [],
        'Pn_St_dU': [],
        'Pn_St_E': [],
        'Pn_St_ee': [],
        'Pn_St_fE': [],
        'Pn_St_fii': [],
        'Pn_St_fl': [],
        'Pn_St_fr': [],
        'Pn_St_fuu': [],
        'Pn_St_fY': [],
        'Pn_St_gaa': [],
        'Pn_St_gee': [],
        'Pn_St_gl': [],
        'Pn_St_gu': [],
        'Pn_St_ha': [],
        'Pn_St_haa': [],
        'Pn_St_hE': [],
        'Pn_St_hI': [],
        'Pn_St_hoo': [],
        'Pn_St_hU': [],
        'Pn_St_In': [],
        'Pn_St_jaa': [],
        'Pn_St_joo': [],
        'Pn_St_k22': [],
        'Pn_St_k9': [],
        'Pn_St_ka': [],
        'Pn_St_kEE': [],
        'Pn_St_kee': [],
        'Pn_St_kl': [],
        'Pn_St_kl': [],
        'Pn_St_kO': [],
        'Pn_St_koo': [],
        'Pn_St_kU': [],
        'Pn_St_kuu': [],
        'Pn_St_l9': [],
        'Pn_St_la': [],
        'Pn_St_laa': [],
        'Pn_St_lee': [],
        'Pn_St_lI': [],
        'Pn_St_lii': [],
        'Pn_St_m22': [],
        'Pn_St_maa': [],
        'Pn_St_mE': [],
        'Pn_St_mI': [],
        'Pn_St_mO': [],
        'Pn_St_moo': [],
        'Pn_St_mU': [],
        'Pn_St_myy': [],
        'Pn_St_myy': [],
        'Pn_St_na': [],
        'Pn_St_naa': [],
        'Pn_St_nE': [],
        'Pn_St_nO': [],
        'Pn_St_nU': [],
        'Pn_St_oo': [],
        'Pn_St_pa': [],
        'Pn_St_paa': [],
        'Pn_St_pI': [],
        'Pn_St_pj': [],
        'Pn_St_pl': [],
        'Pn_St_pO': [],
        'Pn_St_pU': [],
        'Pn_St_pU': [],
        'Pn_St_ra': [],
        'Pn_St_raa': [],
        'Pn_St_ree': [],
        'Pn_St_rI': [],
        'Pn_St_rii': [],
        'Pn_St_rO': [],
        'Pn_St_sl': [],
        'Pn_St_Sa': [],
        'Pn_St_SE': [],
        'Pn_St_SI': [],
        'Pn_St_Sl': [],
        'Pn_St_Sp': [],
        'Pn_St_Sr': [],
        'Pn_St_Sta': [],
        'Pn_St_StE': [],
        'Pn_St_Str': [],
        'Pn_St_Stuu': [],
        'Pn_St_Suu': [],
        'Pn_St_Sv': [],
        'Pn_St_ta': [],
        'Pn_St_taa': [],
        'Pn_St_tE': [],
        'Pn_St_tee': [],
        'Pn_St_tI': [],
        'Pn_St_tii': [],
        'Pn_St_tO': [],
        'Pn_St_tr': [],
        'Pn_St_tr': [],
        'Pn_St_tsa': [],
        'Pn_St_tsaa': [],
        'Pn_St_tsee': [],
        'Pn_St_tsI': [],
        'Pn_St_tsii': [],
        'Pn_St_tsv': [],
        'Pn_St_tU': [],
        'Pn_St_tY': [],
        'Pn_St_U': [],
        'Pn_St_uu': [],
        'Pn_St_va': [],
        'Pn_St_vaa': [],
        'Pn_St_vaa': [],
        'Pn_St_vE': [],
        'Pn_St_vii': [],
        'Pn_St_vO': [],
        'Pn_St_voo': [],
        'Pn_St_vu': [],
        'Pn_St_vY': [],
        'Pn_St_za': [],
        'Pn_St_zaa': [],
        'Pn_St_zE': [],
        'Pn_St_zI': [],
        'Pn_St_zii': [],
        'Pn_St_zO': [],
        'Pn_St_zoo': [],
        'Pn_St_zuu': [],
       }


In [3]:
phonos = {
        # phonological representation of words and similarities at phonological level 
        # Oberbegriffe UND Wörter:
        'Pnw_St_m22_bel': [(inclPhon, 'Pn_St_m22')], 
        'Pnw_St_tsIm_mA': [(inclPhon, 'Pn_St_tsI')],
        'Pnw_St_gee_gen_Stant': [(inclPhon, 'Pn_St_gee')],
        'Pnw_St_klaI_duN': [(inclPhon, 'Pn_St_kl')],
        'Pnw_St_maal_tsaIt': [(inclPhon, 'Pn_St_maa')],
        'Pnw_St_tEE_tIc_kaIt': [(inclPhon, 'Pn_St_tee')],
        'Pnw_St_tiA': [(inclPhon, 'Pn_St_tii')],
        'Pnw_St_E_sen': [(inclPhon, 'Pn_St_E')],
        'Pnw_St_trIN_ken': [(inclPhon, 'Pn_St_tr')],
        'Pnw_St_mEnS': [(inclPhon, 'Pn_St_mE')],
        'Pnw_St_lee_be_St_vee_zen': [(inclPhon, 'Pn_St_lee')],
        'Pnw_St_k9A_pA_St_taIl': [(inclPhon, 'Pn_St_k9')],
        'Pnw_St_tsaIt': [(inclPhon, 'Pn_St_tsa')],
        'Pnw_St_Slaa_fen': [(inclPhon, 'Pn_St_Sl')],
        'Pnw_St_lee_zen': [(inclPhon, 'Pn_St_lee')],
        'Pnw_St_hI_mel': [(inclPhon, 'Pn_St_hI')],
        'Pnw_St_vE_tA': [(inclPhon, 'Pn_St_vE')],
        'Pnw_ge_St_SENk': [(inclPhon, 'Pn_St_SE')],
        'Pnw_Spa_St_tsiA_gaN': [(inclPhon, 'Pn_St_tsii')],
        'Pnw_St_raI_ze': [(inclPhon, 'Pn_St_ra')],
        'Pnw_na_St_tuA': [(inclPhon, 'Pn_St_tU')],
        'Pnw_St_tsaal': [(inclPhon, 'Pn_St_tsaa')],
        'Pnw_St_SraI_ben': [(inclPhon, 'Pn_St_Sr')],
        'Pnw_St_zuu_pA_St_maAkt': [(inclPhon, 'Pn_St_zuu')],
        'Pnw_bE_ke_St_raI': [(inclPhon, 'Pn_St_ra')],
        'Pnw_St_In_tA_nEt': [(inclPhon, 'Pn_St_In')],
        'Pnw_St_m22_bel_haUs': [(inclPhon, 'Pn_St_m22')],
        'Pnw_St_vaa_ren_haUs': [(inclPhon, 'Pn_St_vaa')],
        'Pnw_a_po_St_tee_ke': [(inclPhon, 'Pn_St_tee')],
        
        # Aufstehen / Zubettgehen: 
        'Pnw_St_bEt': [(inclPhon, 'Pn_St_bE')],
        'Pnw_St_dE_ke': [(inclPhon, 'Pn_St_dE')],
        'Pnw_St_vE_kA': [(inclPhon, 'Pn_St_vE')],
        'Pnw_St_baat': [(inclPhon, 'Pn_St_baa')],
        'Pnw_St_zaI_fe': [(inclPhon, 'Pn_St_za')],
        'Pnw_St_han_tuuc': [(inclPhon, 'Pn_St_ha')],
        'Pnw_St_dU_Se': [(inclPhon, 'Pn_St_dU')],
        'Pnw_St_baa_de_St_wan_ne': [(inclPhon, 'Pn_St_baa')],
        'Pnw_St_tsaan_pas_taa': [(inclPhon, 'Pn_St_tsaa')],
        'Pnw_St_tsaan_bYAs_te': [(inclPhon, 'Pn_St_tsaa')],
        'Pnw_St_bE_cA': [(inclPhon, 'Pn_St_bE')],
        'Pnw_St_dee_oo': [(inclPhon, 'Pn_St_dee')],
        'Pnw_St_Stuul': [(inclPhon, 'Pn_St_Stuu')],
        'Pnw_St_klaI_dA_St_SraNk': [(inclPhon, 'Pn_St_Sr')],
        'Pnw_St_byy_gel': [(inclPhon, 'Pn_St_byy')],
        'Pnw_St_hoo_ze': [(inclPhon, 'Pn_St_hoo')],
        'Pnw_St_hEmt': [(inclPhon, 'Pn_St_hE')],
        'Pnw_St_Un_tA_St_hoo_ze': [(inclPhon, 'Pn_St_U')],
        'Pnw_St_slIp': [(inclPhon, 'Pn_St_sl')],
        'Pnw_St_Un_tA_St_hEmt': [(inclPhon, 'Pn_St_U')],
        'Pnw_St_rOk': [(inclPhon, 'Pn_St_rO')],
        'Pnw_St_bluu_ze': [(inclPhon, 'Pn_St_bl')],
        'Pnw_St_trEE_nINs_St_an_tsuuk': [(inclPhon, 'Pn_St_tr')],
        'Pnw_St_nact_hEmt': [(inclPhon, 'Pn_St_na')],
        'Pnw_St_buuc': [(inclPhon, 'Pn_St_buu')],
        'Pnw_St_tsaIt_SrIft': [(inclPhon, 'Pn_St_tsa')],
        'Pnw_St_lam_pe': [(inclPhon, 'Pn_St_la')],
        'Pnw_St_Sal_tA': [(inclPhon, 'Pn_St_Sa')],
        'Pnw_St_hEn_dii': [(inclPhon, 'Pn_St_hE')],
        'Pnw_ta_ST_blE_ten': [(inclPhon, 'Pn_St_bl')],
    
         # Mahlzeiten: 
        'Pnw_St_fryy_StYk': [(inclPhon, 'Pn_St_fr')],
        'Pnw_St_mIt_taak_St_Es_sen': [(inclPhon, 'Pn_St_mI')],
        'Pnw_St_aa_bent_St_Es_sen': [(inclPhon, 'Pn_St_aa')],
        'Pnw_St_tE_lA': [(inclPhon, 'Pn_St_tE')],
        'Pnw_St_ta_se': [(inclPhon, 'Pn_St_ta')],
        'Pnw_St_mE_sA': [(inclPhon, 'Pn_St_mE')],
        'Pnw_St_gaa_bel': [(inclPhon, 'Pn_St_gaa')],
        'Pnw_St_l9_fel': [(inclPhon, 'Pn_St_l9')],
        'Pnw_St_glaas': [(inclPhon, 'Pn_St_gl')],
        'Pnw_St_br22t_cen': [(inclPhon, 'Pn_St_br')],
        'Pnw_St_bUt_tA': [(inclPhon, 'Pn_St_bU')],
        'Pnw_St_aUf_SnIt': [(inclPhon, 'Pn_St_aU')],
        'Pnw_maa_me_St_laa_de': [(inclPhon, 'Pn_St_laa')],
        'Pnw_St_braa_ten': [(inclPhon, 'Pn_St_br')],
        'Pnw_ge_St_myy_ze': [(inclPhon, 'Pn_St_myy')],
        'Pnw_kA_St_tO_feln': [(inclPhon, 'Pn_St_tO')],
        'Pnw_St_raIs': [(inclPhon, 'Pn_St_ra')],
        'Pnw_St_pO_mes': [(inclPhon, 'Pn_St_pO')],
        'Pnw_St_naac_tIS': [(inclPhon, 'Pn_St_naa')],
        'Pnw_St_pU_dIN': [(inclPhon, 'Pn_St_pU')],
        'Pnw_St_joo_guAt': [(inclPhon, 'Pn_St_joo')],
        'Pnw_St_frYc_te': [(inclPhon, 'Pn_St_fr')],
        'Pnw_St_eeAt_bee_re': [(inclPhon, 'Pn_St_ee')],
        'Pnw_St_ap_fel': [(inclPhon, 'Pn_St_a')],
        'Pnw_St_ap_fel_St_zii_ne': [(inclPhon, 'Pn_St_zii')],
        'Pnw_St_man_da_St_rii_ne': [(inclPhon, 'Pn_St_rii')],
        'Pnw_St_rInt': [(inclPhon, 'Pn_St_rI')],
        'Pnw_St_SvaIn': [(inclPhon, 'Pn_St_Sv')],
        'Pnw_St_hyyn_cen': [(inclPhon, 'Pn_St_Sv')],
        'Pnw_St_mIlc': [(inclPhon, 'Pn_St_mI')],
        'Pnw_ge_St_vyA_tse': [(inclPhon, 'Pn_St_vY')],
        'Pnw_St_zalts': [(inclPhon, 'Pn_St_za')],
        'Pnw_St_fE_fA': [(inclPhon, 'Pn_St_fE')],
        'Pnw_za_St_laat': [(inclPhon, 'Pn_St_laa')],
        'Pnw_St_pa_prii_kaa': [(inclPhon, 'Pn_St_pa')],
        'Pnw_St_guA_ke': [(inclPhon, 'Pn_St_gu')],
        'Pnw_to_St_maa_te': [(inclPhon, 'Pn_St_maa')],
        'Pnw_St_vuAst': [(inclPhon, 'Pn_St_vu')],
        'Pnw_za_St_laa_mii': [(inclPhon, 'Pn_St_laa')],
        'Pnw_St_kEE_ze': [(inclPhon, 'Pn_St_kEE')],
        'Pnw_St_kee_kse': [(inclPhon, 'Pn_St_kee')],
        'Pnw_So_ko_St_laa_de': [(inclPhon, 'Pn_St_laa')],
        'Pnw_pra_St_lii_nen': [(inclPhon, 'Pn_St_lii')],
        'Pnw_St_zaa_ne': [(inclPhon, 'Pn_St_zaa')],
        'Pnw_ge_St_hak_tes': [(inclPhon, 'Pn_St_ha')],
        'Pnw_St_flaIS': [(inclPhon, 'Pn_St_fl')],
        'Pnw_St_tiif_kyyl_St_kOst': [(inclPhon, 'Pn_St_tii')],
        'Pnw_lI_St_k2A': [(inclPhon, 'Pn_St_k22')],
        'Pnw_St_zEkt': [(inclPhon, 'Pn_St_zE')],
        'Pnw_St_koo_laa': [(inclPhon, 'Pn_St_koo')],
        'Pnw_St_lI_mo_St_naa_de': [(inclPhon, 'Pn_St_naa')],
        'Pnw_St_va_sA': [(inclPhon, 'Pn_St_va')],
        'Pnw_St_pIt_tsaa': [(inclPhon, 'Pn_St_pI')],
        'Pnw_St_aIn_tOpf': [(inclPhon, 'Pn_St_aI')],
        'Pnw_St_vaIn': [(inclPhon, 'Pn_St_va')],
        'Pnw_St_biA': [(inclPhon, 'Pn_St_bi')],
        'Pnw_St_zaft': [(inclPhon, 'Pn_St_za')],
        'Pnw_St_ka_fee': [(inclPhon, 'Pn_St_ka')],
        'Pnw_St_tee': [(inclPhon, 'Pn_St_tee')],
       
        # Tageszeiten, ...:
        'Pnw_St_mOA_gen': [(inclPhon, 'Pn_St_mO')],
        'Pnw_St_mIt_tac': [(inclPhon, 'Pn_St_mI')],
        'Pnw_St_naac_mIt_tac': [(inclPhon, 'Pn_St_naa')],
        'Pnw_St_aa_bent': [(inclPhon, 'Pn_St_aa')],
        'Pnw_St_nact': [(inclPhon, 'Pn_St_na')],
        'Pnw_St_moont': [(inclPhon, 'Pn_St_moo')],
        'Pnw_St_zO_ne': [(inclPhon, 'Pn_St_zO')],
        'Pnw_St_uuA': [(inclPhon, 'Pn_St_uu')],
        'Pnw_St_taak': [(inclPhon, 'Pn_St_taa')],
        'Pnw_St_vO_ce': [(inclPhon, 'Pn_St_vO')],
        'Pnw_St_moo_naat': [(inclPhon, 'Pn_St_moo')],
        'Pnw_St_jaA': [(inclPhon, 'Pn_St_jaa')],
    
        # draussen: 
        'Pnw_St_vaAm': [(inclPhon, 'Pn_St_vaa')],
        'Pnw_St_kalt': [(inclPhon, 'Pn_St_ka')],
        'Pnw_St_ree_gen': [(inclPhon, 'Pn_St_ree')],
        'Pnw_St_hI_tse': [(inclPhon, 'Pn_St_hI')],
        'Pnw_St_SIAm': [(inclPhon, 'Pn_St_SI')],
        'Pnw_St_Suu_e': [(inclPhon, 'Pn_St_Suu')],
        'Pnw_St_nE_se': [(inclPhon, 'Pn_St_nE')],
        'Pnw_St_byA_gA_St_StaIc': [(inclPhon, 'Pn_St_bY')],
        'Pnw_St_I_nen_St_Stat': [(inclPhon, 'Pn_St_Sta')],
        'Pnw_St_valt': [(inclPhon, 'Pn_St_va')],
        'Pnw_St_paAk': [(inclPhon, 'Pn_St_paa')],
        'Pnw_St_baNk': [(inclPhon, 'Pn_St_ba')],
        'Pnw_St_raa_zen': [(inclPhon, 'Pn_St_raa')],
        'Pnw_St_vii_ze': [(inclPhon, 'Pn_St_vii')],
        'Pnw_St_baUm': [(inclPhon, 'Pn_St_ba')],
        'Pnw_St_haUs': [(inclPhon, 'Pn_St_ha')],
        'Pnw_St_fluuk_haa_fen': [(inclPhon, 'Pn_St_fl')],
        'Pnw_St_baan_hoof': [(inclPhon, 'Pn_St_baa')],
        'Pnw_St_bUs': [(inclPhon, 'Pn_St_bU')],
        'Pnw_St_hal_te_St_StE_le': [(inclPhon, 'Pn_St_StE')],
    
        # drinnen: 
        'Pnw_St_tYA': [(inclPhon, 'Pn_St_tY')],
        'Pnw_St_lIct': [(inclPhon, 'Pn_St_lI')],
        'Pnw_St_tIS': [(inclPhon, 'Pn_St_tI')],
        'Pnw_St_voon_St_tsI_mA': [(inclPhon, 'Pn_St_voo')],
        'Pnw_St_Es_St_tsI_mA': [(inclPhon, 'Pn_St_tsI')],
        'Pnw_St_Slaaf_St_tsI_mA': [(inclPhon, 'Pn_St_Sl')],
        'Pnw_St_fEAn_zee_A': [(inclPhon, 'Pn_St_fE')],
        'Pnw_kOm_St_pjuu_tA': [(inclPhon, 'Pn_St_pj')],
        'Pnw_St_SraNk': [(inclPhon, 'Pn_St_Sr')],
        'Pnw_ree_St_gaal': [(inclPhon, 'Pn_St_gaa')],
        'Pnw_St_kuu_lii': [(inclPhon, 'Pn_St_kuu')],
        'Pnw_St_SraIp_blOk': [(inclPhon, 'Pn_St_Sr')],
        'Pnw_St_briif_pa_St_piA': [(inclPhon, 'Pn_St_br')],
        'Pnw_St_briif_Um_Slaak': [(inclPhon, 'Pn_St_br')],
        'Pnw_St_bUnt_StIft': [(inclPhon, 'Pn_St_bU')],
        'Pnw_St_blaI_StIft': [(inclPhon, 'Pn_St_bl')],
        'Pnw_St_SraIp_hEft': [(inclPhon, 'Pn_St_Sr')],
        'Pnw_St_glyy_St_bIA_ne': [(inclPhon, 'Pn_St_gl')],
    
        # mit Menschen:     
        'Pnw_St_zoon': [(inclPhon, 'Pn_St_zoo')],
        'Pnw_St_tOc_tA': [(inclPhon, 'Pn_St_tO')],
        'Pnw_St_paAt_nA': [(inclPhon, 'Pn_St_paa')],
        'Pnw_St_mIt_be_St_voo_nA': [(inclPhon, 'Pn_St_mI')],
        'Pnw_St_mUnt': [(inclPhon, 'Pn_St_mU')],
        'Pnw_St_aU_ge': [(inclPhon, 'Pn_St_aU')],
        'Pnw_St_haUt': [(inclPhon, 'Pn_St_ha')],
        'Pnw_ge_St_zIct': [(inclPhon, 'Pn_St_zI')],
        'Pnw_St_haa_re': [(inclPhon, 'Pn_St_haa')],
        'Pnw_St_baAt': [(inclPhon, 'Pn_St_baa')],
        'Pnw_St_aAm': [(inclPhon, 'Pn_St_aa')],
        'Pnw_St_baIn': [(inclPhon, 'Pn_St_ba')],
        'Pnw_St_fuus': [(inclPhon, 'Pn_St_fuu')],
    
        # Aktivitäten: 
        'Pnw_St_ree_den': [(inclPhon, 'Pn_St_ree')],
        'Pnw_St_briif': [(inclPhon, 'Pn_St_br')],
        'Pnw_St_Spii_len': [(inclPhon, 'Pn_St_Sp')],
        'Pnw_St_kO_cen': [(inclPhon, 'Pn_St_kO')],
        'Pnw_St_aUs_St_gee_en': [(inclPhon, 'Pn_St_aU')],
        'Pnw_St_hUnt': [(inclPhon, 'Pn_St_hU')],
        'Pnw_St_kat_tse': [(inclPhon, 'Pn_St_ka')],
        'Pnw_St_fY_tAn': [(inclPhon, 'Pn_St_fY')],
        'Pnw_St_StraI_celn': [(inclPhon, 'Pn_St_Str')],
        'Pnw_St_kUs': [(inclPhon, 'Pn_St_kU')],
    
        # Zahlen-Wörter/Wortteile: 
        'Pnw_St_nUl': [(inclPhon, 'Pn_St_nU')],
        'Pnw_St_aInts': [(inclPhon, 'Pn_St_aI')],
        'Pnw_St_tsvaI': [(inclPhon, 'Pn_St_tsv')],
        'Pnw_St_draI': [(inclPhon, 'Pn_St_dr')],
        'Pnw_St_fiA': [(inclPhon, 'Pn_St_fii')],
        'Pnw_St_fYmf': [(inclPhon, 'Pn_St_fY')],
        'Pnw_St_zEks': [(inclPhon, 'Pn_St_zE')],
        'Pnw_St_zii_ben': [(inclPhon, 'Pn_St_zii')],
        'Pnw_St_act': [(inclPhon, 'Pn_St_a')],
        'Pnw_St_nOIn': [(inclPhon, 'Pn_St_nO')],
        'Pnw_St_tseen': [(inclPhon, 'Pn_St_tsee')],
        'Pnw_St_Elf': [(inclPhon, 'Pn_St_E')],
        'Pnw_St_tsv9lf': [(inclPhon, 'Pn_St_tsv')],
        'Pnw_St_zsvan_tsIc': [(inclPhon, 'Pn_St_tsv')],
        'Pnw_tsIc': [(inclPhon, 'Pn_St_tsI')],
        'Pnw_St_hUn_dAt': [(inclPhon, 'Pn_St_hU')],
        'Pnw_St_taU_zent': [(inclPhon, 'Pn_St_ta')],
    
        # Funktionswort 
        'Pnw_St_Unt': [(inclPhon, 'Pn_St_U')],
        'Pnw_St_oo_dA': [(inclPhon, 'Pn_St_oo')],
        'Pnw_St_dEA': [(inclPhon, 'Pn_St_dE')],
        'Pnw_St_dii': [(inclPhon, 'Pn_St_dii')],
        'Pnw_St_das': [(inclPhon, 'Pn_St_da')],
    
        # weiterer Konsum:
        'Pnw_St_plEI_stEI_Sen': [(inclPhon, 'Pn_St_pl')],
        'Pnw_St_pU_pe': [(inclPhon, 'Pn_St_pU')],
        'Pnw_St_bluu_men': [(inclPhon, 'Pn_St_bl')],
        'Pnw_ge_St_SEnk_pa_St_piA': [(inclPhon, 'Pn_St_SE')],
        'Pnw_St_bInt_St_faa_den': [(inclPhon, 'Pn_St_bI')],
        'Pnw_St_klee_be_St_StraI_fen': [(inclPhon, 'Pn_St_kl')],

       }


In [4]:
concepts_deep = {
        # underlying concepts 

        # nur semantische Begrifflichkeiten  
        'Cn_AufstehenSichReinigen': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_SichAnziehen': [(assocWith, 'Cn_Taetigkeit')],

    }


In [5]:
concepts = {
        # Oberbegriffe UND Wörter:
        'Cn_Moebel': [(assocWith, 'Cn_Gegenstand'), (assocWith, 'Cn_Zimmer')],
        'Cn_Zimmer': [(assocWith, 'Cn_Haus')],
        'Cn_Gegenstand': [],
        'Cn_Kleidung': [(assocWith, 'Cn_SichAnziehen')],
        'Cn_Mahlzeit': [(assocWith, 'Cn_Essen')],
        'Cn_Taetigkeit': [],
        'Cn_Tier': [(assocWith, 'Cn_Lebewesen')],
        'Cn_Essen': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_Trinken': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_Mensch': [(assocWith, 'Cn_Lebewesen')],
        'Cn_Lebewesen': [],
        'Cn_Koerperteil': [(assocWith, 'Cn_Lebewesen')],
        'Cn_Zeit': [],
        'Cn_Schlafen': [],
        'Cn_Lesen': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_Himmel': [],
        'Cn_Wetter': [(assocWith, 'Cn_Himmel')],
        'Cn_Geschenk': [],
        'Cn_Spaziergang': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_Reise': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_Natur': [],
        'Cn_Zahl': [],
        'Cn_Schreiben': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_Supermarkt': [],
        'Cn_Baeckerei': [],
        'Cn_Internet': [],
        'Cn_Moebelhaus': [],
        'Cn_Warenhaus': [],
        'Cn_Apotheke': [],

        
        # Aufstehen / Zubettgehen: 
        'Cn_Bett': [(assocWith, 'Cn_Moebel'), (buyIn, 'Cn_Moebelhaus')],
        'Cn_Decke': [(assocWith, 'Cn_Schlafen')],
        'Cn_Wecker': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_Schlafen'), (assocWith, 'Cn_AufstehenSichReinigen')],
        'Cn_Bad': [(assocWith, 'Cn_Zimmer'), (assocWith, 'Cn_AufstehenSichReinigen')],
        'Cn_Seife': [(assocWith, 'Cn_kaufImSupermarkt'), (assocWith, 'Cn_AufstehenSichReinigen')],
        'Cn_Handtuch': [(assocWith, 'Cn_AufstehenSichReinigen')],
        'Cn_Dusche': [(assocWith, 'Cn_AufstehenSichReinigen')],
        'Cn_Badewanne': [(assocWith, 'Cn_AufstehenSichReinigen')],
        'Cn_Zahnpasta': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_AufstehenSichReinigen')],
        'Cn_Zahnbuerste': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_AufstehenSichReinigen')],
        'Cn_Becher': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_AufstehenSichReinigen')],
        'Cn_Deo': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_AufstehenSichReinigen')],
        'Cn_Stuhl': [(assocWith, 'Cn_Moebel')],
        'Cn_Kleiderschrank': [(buyIn, 'Cn_Moebelhaus'), (assocWith, 'Cn_Moebel'), (assocWith, 'Cn_SichAnziehen')],
        'Cn_Buegel': [(assocWith, 'Cn_SichAnziehen')],
        'Cn_Hose': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_Kleidung')],
        'Cn_Hemd': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_Kleidung')],
        'Cn_Unterhose': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_Kleidung')],
        'Cn_Slip': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_Kleidung')],
        'Cn_Unterhemd': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_Kleidung')],
        'Cn_Rock': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_Kleidung')],
        'Cn_Bluse': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_Kleidung')],
        'Cn_Trainingsanzug': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_Kleidung')],
        'Cn_Nachthemd': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_Kleidung')],
        'Cn_Buch': [(buyIn, 'Cn_Internet'), (assocWith, 'Cn_Lesen')],
        'Cn_Zeitschrift': [(buyIn, 'Cn_Supermarkt')],
        'Cn_Lampe': [(buyIn, 'Cn_Moebelhaus'), (assocWith, 'Cn_Lesen')],
        'Cn_Schalter': [(assocWith, 'Cn_Lesen')],
        'Cn_Handy': [(buyIn, 'Cn_Internet'), (assocWith, 'Cn_Lesen')],
        'Cn_Tabletten': [(buyIn, 'Cn_Apotheke'), (assocWith, 'Cn_Schlafen')],
    
         # essen, trinken: 
        'Cn_Fruehstueck': [(assocWith, 'Cn_Mahlzeit')],
        'Cn_Mittagessen': [(assocWith, 'Cn_Mahlzeit')],
        'Cn_Abendessen': [(assocWith, 'Cn_Mahlzeit')],
        'Cn_Teller': [(assocWith, 'Cn_Essen')],
        'Cn_Tasse': [(assocWith, 'Cn_Trinken')],
        'Cn_Messer': [(assocWith, 'Cn_Essen')],
        'Cn_Gabel': [(assocWith, 'Cn_Essen')],
        'Cn_Loeffel': [(assocWith, 'Cn_Essen')],
        'Cn_Glas': [(assocWith, 'Cn_Trinken')],
        'Cn_Broetchen': [(buyIn, 'Cn_Baeckerei'), (assocWith, 'Cn_Essen')],
        'Cn_Butter': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Aufschnitt': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Marmelade': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Braten': [(assocWith, 'Cn_Essen')],
        'Cn_Gemuese': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Kartoffeln': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Reis': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Pommes': [(assocWith, 'Cn_Essen')],
        'Cn_Nachtisch': [(assocWith, 'Cn_Essen')],
        'Cn_Pudding': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Jogurt': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Fruechte': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Erdbeere': [(assocWith, 'Cn_Essen')],
        'Cn_Apfel': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Apfelsine': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Mandarine': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Rind': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Tier'), (assocWith, 'Cn_Essen')],
        'Cn_Schwein': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Tier'), (assocWith, 'Cn_Essen')],
        'Cn_Huehnchen': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Tier'), (assocWith, 'Cn_Essen')],
        'Cn_Milch': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen'), (assocWith, 'Cn_Trinken')],
        'Cn_Gewuerze': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Salz': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen'), (assocWith, 'Cn_Gewuerze')],
        'Cn_Pfeffer': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen'), (assocWith, 'Cn_Gewuerze')],
        'Cn_Salat': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Paprika': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Gurke': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Tomate': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Wurst': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Salami': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Kaese': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Kekse': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Schokolade': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Pralinen': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Sahne': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Gehacktes': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Fleisch': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Tiefkuehlkost': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Likoer': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Trinken')],
        'Cn_Sekt': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Trinken')],
        'Cn_Kola': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Trinken')],
        'Cn_Limonade': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Trinken')],
        'Cn_Wasser': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Trinken')],
        'Cn_Pizza': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Eintopf': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Essen')],
        'Cn_Wein': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Trinken')],
        'Cn_Bier': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Trinken')],
        'Cn_Saft': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Trinken')],
        'Cn_Kaffee': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Trinken')],
        'Cn_Tee': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Trinken')],
       
        # Tageszeiten, ...:
        'Cn_Morgen': [(assocWith, 'Cn_Zeit')],
        'Cn_Mittag': [(assocWith, 'Cn_Zeit')],
        'Cn_Nachmittag': [(assocWith, 'Cn_Zeit')],
        'Cn_Abend': [(assocWith, 'Cn_Zeit')],
        'Cn_Nacht': [(assocWith, 'Cn_Zeit')],
        'Cn_Mond': [(assocWith, 'Cn_Himmel')],
        'Cn_Sonne': [(assocWith, 'Cn_Himmel')],
        'Cn_Uhr': [(assocWith, 'Cn_Zeit')],
        'Cn_Tag': [(assocWith, 'Cn_Zeit')],
        'Cn_Woche': [(assocWith, 'Cn_Zeit')],
        'Cn_Monat': [(assocWith, 'Cn_Zeit')],
        'Cn_Jahr': [(assocWith, 'Cn_Zeit')],
    
        # draussen: 
        'Cn_Warm': [(assocWith, 'Cn_Wetter')],
        'Cn_Kalt': [(assocWith, 'Cn_Wetter')],
        'Cn_Regen': [(assocWith, 'Cn_Wetter')],
        'Cn_Hitze': [(assocWith, 'Cn_Wetter')],
        'Cn_Schirm': [(assocWith, 'Cn_Wetter')],
        'Cn_Schuhe': [(assocWith, 'Cn_Kleidung')],
        'Cn_Naesse': [(assocWith, 'Cn_Wetter')],
        'Cn_Buergersteig': [(assocWith, 'Cn_Spaziergang')],
        'Cn_Innenstadt': [(assocWith, 'Cn_Spaziergang')],
        'Cn_Wald': [(assocWith, 'Cn_Spaziergang'), (assocWith, 'Cn_Natur')],
        'Cn_Park': [(assocWith, 'Cn_Spaziergang')],
        'Cn_Bank': [(assocWith, 'Cn_Spaziergang')],
        'Cn_Rasen': [(assocWith, 'Cn_Spaziergang')],
        'Cn_Wiese': [(assocWith, 'Cn_Spaziergang'), (assocWith, 'Cn_Natur')],
        'Cn_Baum': [(assocWith, 'Cn_Spaziergang'), (assocWith, 'Cn_Natur')],
        'Cn_Haus': [(assocWith, 'Cn_Spaziergang')],
        'Cn_Flughafen': [(assocWith, 'Cn_Reise')],
        'Cn_Bahnhof': [(assocWith, 'Cn_Reise')],
        'Cn_Bus': [(assocWith, 'Cn_Reise')],
        'Cn_Haltestelle': [(assocWith, 'Cn_Reise')],
    
        # drinnen: 
        'Cn_Tuer': [(assocWith, 'Cn_Haus')],
        'Cn_Licht': [(assocWith, 'Cn_Haus')],
        'Cn_Tisch': [(buyIn, 'Cn_Moebelhaus'), (assocWith, 'Cn_Moebel')],
        'Cn_Wohnzimmer': [(assocWith, 'Cn_Zimmer')],
        'Cn_Esszimmer': [(assocWith, 'Cn_Zimmer')],
        'Cn_Schlafzimmer': [(assocWith, 'Cn_Zimmer')],
        'Cn_Fernseher': [(buyIn, 'Cn_Internet'), (assocWith, 'Cn_Taetigkeit')],
        'Cn_Computer': [(buyIn, 'Cn_Internet'), (assocWith, 'Cn_Taetigkeit')],
        'Cn_Schrank': [(buyIn, 'Cn_Moebelhaus'), (assocWith, 'Cn_Moebel')],
        'Cn_Regal': [(buyIn, 'Cn_Moebelhaus'), (assocWith, 'Cn_Moebel')],
        'Cn_Kuli': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Schreiben')],
        'Cn_Schreibblock': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Schreiben')],
        'Cn_Briefpapier': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Schreiben')],
        'Cn_Briefumschlag': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Schreiben')],
        'Cn_Buntstift': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Schreiben')],
        'Cn_Bleistift': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Schreiben')],
        'Cn_Schreibheft': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Schreiben')],
        'Cn_Gluehbirne': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Lampe')],
    
        # mit Menschen:     
        'Cn_Sohn': [(assocWith, 'Cn_Mensch')],
        'Cn_Tochter': [(assocWith, 'Cn_Mensch')],
        'Cn_Partner': [(assocWith, 'Cn_Mensch')],
        'Cn_Mitbewohner': [(assocWith, 'Cn_Mensch')],
        'Cn_Mund': [(assocWith, 'Cn_Koerperteil')],
        'Cn_Auge': [(assocWith, 'Cn_Koerperteil')],
        'Cn_Haut': [(assocWith, 'Cn_Koerperteil')],
        'Cn_Gesicht': [(assocWith, 'Cn_Koerperteil')],
        'Cn_Haare': [(assocWith, 'Cn_Koerperteil')],
        'Cn_Bart': [(assocWith, 'Cn_Koerperteil')],
        'Cn_Arm': [(assocWith, 'Cn_Koerperteil')],
        'Cn_Bein': [(assocWith, 'Cn_Koerperteil')],
        'Cn_Fuss': [(assocWith, 'Cn_Koerperteil')],
    
        # Aktivitäten: 
        'Cn_Reden': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_Brief': [(assocWith, 'Cn_Lesen')],
        'Cn_Spielen': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_Kochen': [(assocWith, 'Cn_Taetigkeit'), (assocWith, 'Cn_Essen')],
        'Cn_Ausgehen': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_Hund': [(assocWith, 'Cn_Tier')],
        'Cn_Katze': [(assocWith, 'Cn_Tier')],
        'Cn_Fuettern': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_Streicheln': [(assocWith, 'Cn_Taetigkeit')],
        'Cn_Kuss': [(assocWith, 'Cn_Taetigkeit')],
    
        # Zahlen-Wörter/Wortteile: 
        'Cn_Null': [(assocWith, 'Cn_Zahl')],
        'Cn_Eins': [(assocWith, 'Cn_Zahl')],
        'Cn_Zwei': [(assocWith, 'Cn_Zahl')],
        'Cn_Drei': [(assocWith, 'Cn_Zahl')],
        'Cn_Vier': [(assocWith, 'Cn_Zahl')],
        'Cn_Fuenf': [(assocWith, 'Cn_Zahl')],
        'Cn_Sechs': [(assocWith, 'Cn_Zahl')],
        'Cn_Sieben': [(assocWith, 'Cn_Zahl')],
        'Cn_Acht': [(assocWith, 'Cn_Zahl')],
        'Cn_Neun': [(assocWith, 'Cn_Zahl')],
        'Cn_Zehn': [(assocWith, 'Cn_Zahl')],
        'Cn_Elf': [(assocWith, 'Cn_Zahl')],
        'Cn_Zwoelf': [(assocWith, 'Cn_Zahl')],
        'Cn_Zwanzig': [(assocWith, 'Cn_Zahl')],
        'Cn_Zig': [],
        'Cn_Hundert': [(assocWith, 'Cn_Zahl')],
        'Cn_Tausend': [(assocWith, 'Cn_Zahl')],
    
        # Funktionswort 
        'Cn_Und': [],
        'Cn_Oder': [],
        'Cn_Der': [],
        'Cn_Die': [],
        'Cn_Das': [],
    
        # weiterer Konsum:
        'Cn_Playstation': [(buyIn, 'Cn_Internet'), (assocWith, 'Cn_Geschenk')],
        'Cn_Puppe': [(buyIn, 'Cn_Warenhaus'), (assocWith, 'Cn_Geschenk')],
        'Cn_Blumen': [(assocWith, 'Cn_Geschenk'), (assocWith, 'Cn_Natur')],
        'Cn_Geschenkpapier': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Geschenk')],
        'Cn_Bindfaden': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Geschenk')],
        'Cn_Klebestreifen': [(buyIn, 'Cn_Supermarkt'), (assocWith, 'Cn_Geschenk')],

         }


In [6]:
# ----------------------------------------------------------------------------------------------------------------------
spn_concept_keys = [  # one-dim list!!!
        # Oberbegriffe UND Wörter:
        'Cn_Moebel', 
        'Cn_Zimmer',
        'Cn_Gegenstand',
        'Cn_Kleidung',
        'Cn_Mahlzeit',
        'Cn_Taetigkeit',
        'Cn_Tier',
        'Cn_Essen',
        'Cn_Trinken',
        'Cn_Mensch',
        'Cn_Lebewesen',
        'Cn_Koerperteil',
        'Cn_Zeit',
        'Cn_Schlafen',
        'Cn_Lesen',
        'Cn_Himmel',
        'Cn_Wetter',
        'Cn_Geschenk',
        'Cn_Spaziergang',
        'Cn_Reise',
        'Cn_Natur',
        'Cn_Zahl',
        'Cn_Schreiben',
        'Cn_Supermarkt',
        'Cn_Baeckerei',
        'Cn_Internet',
        'Cn_Moebelhaus',
        'Cn_Warenhaus',
        'Cn_Apotheke',

        # Aufstehen / Zubettgehen: 
        'Cn_Bett',
        'Cn_Decke',
        'Cn_Wecker',
        'Cn_Bad',
        'Cn_Seife',
        'Cn_Handtuch',
        'Cn_Dusche',
        'Cn_Badewanne',
        'Cn_Zahnpasta',
        'Cn_Zahnbuerste',
        'Cn_Becher',
        'Cn_Deo',
        'Cn_Stuhl',
        'Cn_Kleiderschrank',
        'Cn_Buegel',
        'Cn_Hose',
        'Cn_Hemd',
        'Cn_Unterhose',
        'Cn_Slip',
        'Cn_Unterhemd',
        'Cn_Rock',
        'Cn_Bluse',
        'Cn_Trainingsanzug',
        'Cn_Nachthemd',
        'Cn_Buch',
        'Cn_Zeitschrift',
        'Cn_Lampe',
        'Cn_Schalter',
        'Cn_Handy',
        'Cn_Tabletten',
    
         # Mahlzeiten: 
        'Cn_Fruehstueck',
        'Cn_Mittagessen',
        'Cn_Abendessen',
        'Cn_Teller',
        'Cn_Tasse',
        'Cn_Messer',
        'Cn_Gabel',
        'Cn_Loeffel',
        'Cn_Glas',
        'Cn_Broetchen',
        'Cn_Butter',
        'Cn_Aufschnitt',
        'Cn_Marmelade',
        'Cn_Braten',
        'Cn_Gemuese',
        'Cn_Kartoffeln',
        'Cn_Reis',
        'Cn_Pommes',
        'Cn_Nachtisch',
        'Cn_Pudding',
        'Cn_Jogurt',
        'Cn_Fruechte',
        'Cn_Erdbeere',
        'Cn_Apfel',
        'Cn_Apfelsine',
        'Cn_Mandarine',
        'Cn_Rind',
        'Cn_Schwein',
        'Cn_Huehnchen',
        'Cn_Milch',
        'Cn_Gewuerze',
        'Cn_Salz',
        'Cn_Pfeffer',
        'Cn_Salat',
        'Cn_Paprika',
        'Cn_Gurke',
        'Cn_Tomate',
        'Cn_Wurst',
        'Cn_Salami',
        'Cn_Kaese',
        'Cn_Kekse',
        'Cn_Schokolade',
        'Cn_Pralinen',
        'Cn_Sahne',
        'Cn_Gehacktes',
        'Cn_Fleisch',
        'Cn_Tiefkuehlkost',
        'Cn_Likoer',
        'Cn_Sekt',
        'Cn_Kola',
        'Cn_Limonade',
        'Cn_Wasser',
        'Cn_Pizza',
        'Cn_Eintopf',
        'Cn_Wein',
        'Cn_Bier',
        'Cn_Saft',
        'Cn_Kaffee',
        'Cn_Tee',
       
        # Tageszeiten, ...:
        'Cn_Morgen',
        'Cn_Mittag',
        'Cn_Nachmittag',
        'Cn_Abend',
        'Cn_Nacht',
        'Cn_Mond',
        'Cn_Sonne',
        'Cn_Uhr',
        'Cn_Tag',
        'Cn_Woche',
        'Cn_Monat',
        'Cn_Jahr',
    
        # draussen: 
        'Cn_Warm',
        'Cn_Kalt',
        'Cn_Regen',
        'Cn_Hitze',
        'Cn_Schirm',
        'Cn_Schuhe',
        'Cn_Naesse',
        'Cn_Buergersteig',
        'Cn_Innenstadt',
        'Cn_Wald',
        'Cn_Park',
        'Cn_Bank',
        'Cn_Rasen',
        'Cn_Wiese',
        'Cn_Baum',
        'Cn_Haus',
        'Cn_Flughafen',
        'Cn_Bahnhof',
        'Cn_Bus',
        'Cn_Haltestelle',
    
        # drinnen: 
        'Cn_Tuer',
        'Cn_Licht',
        'Cn_Tisch',
        'Cn_Wohnzimmer',
        'Cn_Esszimmer',
        'Cn_Schlafzimmer',
        'Cn_Fernseher',
        'Cn_Computer',
        'Cn_Schrank',
        'Cn_Regal',
        'Cn_Kuli',
        'Cn_Schreibblock',
        'Cn_Briefpapier',
        'Cn_Briefumschlag',
        'Cn_Buntstift',
        'Cn_Bleistift',
        'Cn_Schreibheft',
        'Cn_Gluehbirne',
    
        # mit Menschen:     
        'Cn_Sohn',
        'Cn_Tochter',
        'Cn_Partner',
        'Cn_Mitbewohner',
        'Cn_Mund',
        'Cn_Auge',
        'Cn_Haut',
        'Cn_Gesicht',
        'Cn_Haare',
        'Cn_Bart',
        'Cn_Arm',
        'Cn_Bein',
        'Cn_Fuss',
    
        # Aktivitäten: 
        'Cn_Reden',
        'Cn_Brief',
        'Cn_Spielen',
        'Cn_Kochen',
        'Cn_Ausgehen',
        'Cn_Hund',
        'Cn_Katze',
        'Cn_Fuettern',
        'Cn_Streicheln',
        'Cn_Kuss',
    
        # Zahlen-Wörter/Wortteile: 
        'Cn_Null',
        'Cn_Eins',
        'Cn_Zwei',
        'Cn_Drei',
        'Cn_Vier',
        'Cn_Fuenf',
        'Cn_Sechs',
        'Cn_Sieben',
        'Cn_Acht',
        'Cn_Neun',
        'Cn_Zehn',
        'Cn_Elf',
        'Cn_Zwoelf',
        'Cn_Zwanzig',
        'Cn_Zig',
        'Cn_Hundert',
        'Cn_Tausend',
    
        # Funktionswort 
        'Cn_Und',
        'Cn_Oder',
        'Cn_Der',
        'Cn_Die',
        'Cn_Das',
    
        # weiterer Konsum:
        'Cn_Playstation',
        'Cn_Puppe',
        'Cn_Blumen',
        'Cn_Geschenkpapier',
        'Cn_Bindfaden',
        'Cn_Klebestreifen',
    
]


In [7]:
concept_keys = [  # one-dim list!!!
        # Oberbegriffe UND Wörter:
        'C_Moebel', 
        'C_Zimmer',
        'C_Gegenstand',
        'C_Kleidung',
        'C_Mahlzeit',
        'C_Taetigkeit',
        'C_Tier',
        'C_Essen',
        'C_Trinken',
        'C_Mensch',
        'C_Lebewesen',
        'C_Koerperteil',
        'C_Zeit',
        'C_Schlafen',
        'C_Lesen',
        'C_Himmel',
        'C_Wetter',
        'C_Geschenk',
        'C_Spaziergang',
        'C_Reise',
        'C_Natur',
        'C_Zahl',
        'C_Schreiben',
        'C_Supermarkt',
        'C_Baeckerei',
        'C_Internet',
        'C_Moebelhaus',
        'C_Warenhaus',
        'C_Apotheke',

        # Aufstehen / Zubettgehen: 
        'C_Bett',
        'C_Decke',
        'C_Wecker',
        'C_Bad',
        'C_Seife',
        'C_Handtuch',
        'C_Dusche',
        'C_Badewanne',
        'C_Zahnpasta',
        'C_Zahnbuerste',
        'C_Becher',
        'C_Deo',
        'C_Stuhl',
        'C_Kleiderschrank',
        'C_Buegel',
        'C_Hose',
        'C_Hemd',
        'C_Unterhose',
        'C_Slip',
        'C_Unterhemd',
        'C_Rock',
        'C_Bluse',
        'C_Trainingsanzug',
        'C_Nachthemd',
        'C_Buch',
        'C_Zeitschrift',
        'C_Lampe',
        'C_Schalter',
        'C_Handy',
        'C_Tabletten',
    
         # Mahlzeiten: 
        'C_Fruehstueck',
        'C_Mittagessen',
        'C_Abendessen',
        'C_Teller',
        'C_Tasse',
        'C_Messer',
        'C_Gabel',
        'C_Loeffel',
        'C_Glas',
        'C_Broetchen',
        'C_Butter',
        'C_Aufschnitt',
        'C_Marmelade',
        'C_Braten',
        'C_Gemuese',
        'C_Kartoffeln',
        'C_Reis',
        'C_Pommes',
        'C_Nachtisch',
        'C_Pudding',
        'C_Jogurt',
        'C_Fruechte',
        'C_Erdbeere',
        'C_Apfel',
        'C_Apfelsine',
        'C_Mandarine',
        'C_Rind',
        'C_Schwein',
        'C_Huehnchen',
        'C_Milch',
        'C_Gewuerze',
        'C_Salz',
        'C_Pfeffer',
        'C_Salat',
        'C_Paprika',
        'C_Gurke',
        'C_Tomate',
        'C_Wurst',
        'C_Salami',
        'C_Kaese',
        'C_Kekse',
        'C_Schokolade',
        'C_Pralinen',
        'C_Sahne',
        'C_Gehacktes',
        'C_Fleisch',
        'C_Tiefkuehlkost',
        'C_Likoer',
        'C_Sekt',
        'C_Kola',
        'C_Limonade',
        'C_Wasser',
        'C_Pizza',
        'C_Eintopf',
        'C_Wein',
        'C_Bier',
        'C_Saft',
        'C_Kaffee',
        'C_Tee',
       
        # Tageszeiten, ...:
        'C_Morgen',
        'C_Mittag',
        'C_Nachmittag',
        'C_Abend',
        'C_Nacht',
        'C_Mond',
        'C_Sonne',
        'C_Uhr',
        'C_Tag',
        'C_Woche',
        'C_Monat',
        'C_Jahr',
    
        # draussen: 
        'C_Warm',
        'C_Kalt',
        'C_Regen',
        'C_Hitze',
        'C_Schirm',
        'C_Schuhe',
        'C_Naesse',
        'C_Buergersteig',
        'C_Innenstadt',
        'C_Wald',
        'C_Park',
        'C_Bank',
        'C_Rasen',
        'C_Wiese',
        'C_Baum',
        'C_Haus',
        'C_Flughafen',
        'C_Bahnhof',
        'C_Bus',
        'C_Haltestelle',
    
        # drinnen: 
        'C_Tuer',
        'C_Licht',
        'C_Tisch',
        'C_Wohnzimmer',
        'C_Esszimmer',
        'C_Schlafzimmer',
        'C_Fernseher',
        'C_Computer',
        'C_Schrank',
        'C_Regal',
        'C_Kuli',
        'C_Schreibblock',
        'C_Briefpapier',
        'C_Briefumschlag',
        'C_Buntstift',
        'C_Bleistift',
        'C_Schreibheft',
        'C_Gluehbirne',
    
        # mit Menschen:     
        'C_Sohn',
        'C_Tochter',
        'C_Partner',
        'C_Mitbewohner',
        'C_Mund',
        'C_Auge',
        'C_Haut',
        'C_Gesicht',
        'C_Haare',
        'C_Bart',
        'C_Arm',
        'C_Bein',
        'C_Fuss',
    
        # Aktivitäten: 
        'C_Reden',
        'C_Brief',
        'C_Spielen',
        'C_Kochen',
        'C_Ausgehen',
        'C_Hund',
        'C_Katze',
        'C_Fuettern',
        'C_Streicheln',
        'C_Kuss',
    
        # Zahlen-Wörter/Wortteile: 
        'C_Null',
        'C_Eins',
        'C_Zwei',
        'C_Drei',
        'C_Vier',
        'C_Fuenf',
        'C_Sechs',
        'C_Sieben',
        'C_Acht',
        'C_Neun',
        'C_Zehn',
        'C_Elf',
        'C_Zwoelf',
        'C_Zwanzig',
        'C_Zig',
        'C_Hundert',
        'C_Tausend',
    
        # Funktionswort 
        'C_Und',
        'C_Oder',
        'C_Der',
        'C_Die',
        'C_Das',
    
        # weiterer Konsum:
        'C_Playstation',
        'C_Puppe',
        'C_Blumen',
        'C_Geschenkpapier',
        'C_Bindfaden',
        'C_Klebestreifen',
]


concept_digits_keys = [
        'C_Null',
        'C_Eins',
        'C_Zwei',
        'C_Drei',
        'C_Vier',
        'C_Fuenf',
        'C_Sechs',
        'C_Sieben',
        'C_Acht',
        'C_Neun',
]

  

In [8]:
word_keys = [  # one-dim list!!!
        # orthographic representations (word node)

        # Oberbegriffe UND Wörter: 29
        'W_Moebel', 
        'W_Zimmer',
        'W_Gegenstand',
        'W_Kleidung',
        'W_Mahlzeit',
        'W_Taetigkeit',
        'W_Tier',
        'W_Essen',
        'W_Trinken',
        'W_Mensch',
        'W_Lebewesen',
        'W_Koerperteil',
        'W_Zeit',
        'W_Schlafen',
        'W_Lesen',
        'W_Himmel',
        'W_Wetter',
        'W_Geschenk',
        'W_Spaziergang',
        'W_Reise',
        'W_Natur',
        'W_Zahl',
        'W_Schreiben',
        'W_Supermarkt',
        'W_Baeckerei',
        'W_Internet',
        'W_Moebelhaus',
        'W_Warenhaus',
        'W_Apotheke',

        
        # Aufstehen / Zubettgehen: 30
        'W_Bett',
        'W_Decke',
        'W_Wecker',
        'W_Bad',
        'W_Seife',
        'W_Handtuch',
        'W_Dusche',
        'W_Badewanne',
        'W_Zahnpasta',
        'W_Zahnbuerste',
        'W_Becher',
        'W_Deo',
        'W_Stuhl',
        'W_Kleiderschrank',
        'W_Buegel',
        'W_Hose',
        'W_Hemd',
        'W_Unterhose',
        'W_Slip',
        'W_Unterhemd',
        'W_Rock',
        'W_Bluse',
        'W_Trainingsanzug',
        'W_Nachthemd',
        'W_Buch',
        'W_Zeitschrift',
        'W_Lampe',
        'W_Schalter',
        'W_Handy',
        'W_Tabletten',
    
         # Mahlzeiten: 59
        'W_Fruehstueck',
        'W_Mittagessen',
        'W_Abendessen',
        'W_Teller',
        'W_Tasse',
        'W_Messer',
        'W_Gabel',
        'W_Loeffel',
        'W_Glas',
        'W_Broetchen',
        'W_Butter',
        'W_Aufschnitt',
        'W_Marmelade',
        'W_Braten',
        'W_Gemuese',
        'W_Kartoffeln',
        'W_Reis',
        'W_Pommes',
        'W_Nachtisch',
        'W_Pudding',
        'W_Jogurt',
        'W_Fruechte',
        'W_Erdbeere',
        'W_Apfel',
        'W_Apfelsine',
        'W_Mandarine',
        'W_Rind',
        'W_Schwein',
        'W_Huehnchen',
        'W_Milch',
        'W_Gewuerze',
        'W_Salz',
        'W_Pfeffer',
        'W_Salat',
        'W_Paprika',
        'W_Gurke',
        'W_Tomate',
        'W_Wurst',
        'W_Salami',
        'W_Kaese',
        'W_Kekse',
        'W_Schokolade',
        'W_Pralinen',
        'W_Sahne',
        'W_Gehacktes',
        'W_Fleisch',
        'W_Tiefkuehlkost',
        'W_Likoer',
        'W_Sekt',
        'W_Kola',
        'W_Limonade',
        'W_Wasser',
        'W_Pizza',
        'W_Eintopf',
        'W_Wein',
        'W_Bier',
        'W_Saft',
        'W_Kaffee',
        'W_Tee',
       
        # Tageszeiten, ...: 12
        'W_Morgen',
        'W_Mittag',
        'W_Nachmittag',
        'W_Abend',
        'W_Nacht',
        'W_Mond',
        'W_Sonne',
        'W_Uhr',
        'W_Tag',
        'W_Woche',
        'W_Monat',
        'W_Jahr',
    
        # draussen: 20
        'W_Warm',
        'W_Kalt',
        'W_Regen',
        'W_Hitze',
        'W_Schirm',
        'W_Schuhe',
        'W_Naesse',
        'W_Buergersteig',
        'W_Innenstadt',
        'W_Wald',
        'W_Park',
        'W_Bank',
        'W_Rasen',
        'W_Wiese',
        'W_Baum',
        'W_Haus',
        'W_Flughafen',
        'W_Bahnhof',
        'W_Bus',
        'W_Haltestelle',
    
        # drinnen: 18
        'W_Tuer',
        'W_Licht',
        'W_Tisch',
        'W_Wohnzimmer',
        'W_Esszimmer',
        'W_Schlafzimmer',
        'W_Fernseher',
        'W_Computer',
        'W_Schrank',
        'W_Regal',
        'W_Kuli',
        'W_Schreibblock',
        'W_Briefpapier',
        'W_Briefumschlag',
        'W_Buntstift',
        'W_Bleistift',
        'W_Schreibheft',
        'W_Gluehbirne',
    
        # mit Menschen:  13   
        'W_Sohn',
        'W_Tochter',
        'W_Partner',
        'W_Mitbewohner',
        'W_Mund',
        'W_Auge',
        'W_Haut',
        'W_Gesicht',
        'W_Haare',
        'W_Bart',
        'W_Arm',
        'W_Bein',
        'W_Fuss',
    
        # Aktivitäten: 10
        'W_Reden',
        'W_Brief',
        'W_Spielen',
        'W_Kochen',
        'W_Ausgehen',
        'W_Hund',
        'W_Katze',
        'W_Fuettern',
        'W_Streicheln',
        'W_Kuss',
    
        # Zahlen-Wörter/Wortteile: 17
        'W_Null',
        'W_Eins',
        'W_Zwei',
        'W_Drei',
        'W_Vier',
        'W_Fuenf',
        'W_Sechs',
        'W_Sieben',
        'W_Acht',
        'W_Neun',
        'W_Zehn',
        'W_Elf',
        'W_Zwoelf',
        'W_Zwanzig',
        'W_Zig',
        'W_Hundert',
        'W_Tausend',
    
        # Funktionswörter: 5
        'W_Und',
        'W_Oder',
        'W_Der',
        'W_Die',
        'W_Das',
    
        # weiterer Konsum: 6
        'W_Playstation',
        'W_Puppe',
        'W_Blumen',
        'W_Geschenkpapier',
        'W_Bindfaden',
        'W_Klebestreifen',
]

word_digits_keys = [
        'W_Null',
        'W_Eins',
        'W_Zwei',
        'W_Drei',
        'W_Vier',
        'W_Fuenf',
        'W_Sechs',
        'W_Sieben',
        'W_Acht',
        'W_Neun',
]



In [9]:
spn_phono_keys = [  # one-dim list!!! for words, not syllables
        # Oberbegriffe UND Wörter: 29
        'Pnw_St_m22_bel', 
        'Pnw_St_tsIm_mA',
        'Pnw_St_gee_gen_Stant',
        'Pnw_St_klaI_duN',
        'Pnw_St_maal_tsaIt',
        'Pnw_St_tEE_tIc_kaIt',
        'Pnw_St_tiA',
        'Pnw_St_E_sen',
        'Pnw_St_trIN_ken',
        'Pnw_St_mEnS',
        'Pnw_St_lee_be_St_vee_zen',
        'Pnw_St_k9A_pA_St_taIl',
        'Pnw_St_tsaIt',
        'Pnw_St_Slaa_fen',
        'Pnw_St_lee_zen',
        'Pnw_St_hI_mel',
        'Pnw_St_vE_tA',
        'Pnw_ge_St_SENk',
        'Pnw_Spa_St_tsiA_gaN',
        'Pnw_St_raI_ze',
        'Pnw_na_St_tuA',
        'Pnw_St_tsaal',
        'Pnw_St_SraI_ben',
        'Pnw_St_zuu_pA_St_maAkt',
        'Pnw_bE_ke_St_raI',
        'Pnw_St_In_tA_nEt',
        'Pnw_St_m22_bel_haUs',
        'Pnw_St_vaa_ren_haUs',
        'Pnw_a_po_St_tee_ke',
    
        # Aufstehen / Zubettgehen: 
        'Pnw_St_bEt',
        'Pnw_St_dE_ke',
        'Pnw_St_vE_kA',
        'Pnw_St_baat',
        'Pnw_St_zaI_fe',
        'Pnw_St_han_tuuc',
        'Pnw_St_dU_Se',
        'Pnw_St_baa_de_St_wan_ne',
        'Pnw_St_tsaan_pas_taa',
        'Pnw_St_tsaan_bYAs_te',
        'Pnw_St_bE_cA',
        'Pnw_St_dee_oo',
        'Pnw_St_Stuul',
        'Pnw_St_klaI_dA_St_SraNk',
        'Pnw_St_byy_gel',
        'Pnw_St_hoo_ze',
        'Pnw_St_hEmt',
        'Pnw_St_Un_tA_St_hoo_ze',
        'Pnw_St_slIp',
        'Pnw_St_Un_tA_St_hEmt',
        'Pnw_St_rOk',
        'Pnw_St_bluu_ze',
        'Pnw_St_trEE_nINs_St_an_tsuuk',
        'Pnw_St_nact_hEmt',
        'Pnw_St_buuc',
        'Pnw_St_tsaIt_SrIft',
        'Pnw_St_lam_pe',
        'Pnw_St_Sal_tA',
        'Pnw_St_hEn_dii',
        'Pnw_ta_St_blE_ten',
    
         # Mahlzeiten: 
        'Pnw_St_fryy_StYk',
        'Pnw_St_mIt_taak_St_Es_sen',
        'Pnw_St_aa_bent_St_Es_sen',
        'Pnw_St_tE_lA',
        'Pnw_St_ta_se',
        'Pnw_St_mE_sA',
        'Pnw_St_gaa_bel',
        'Pnw_St_l9_fel',
        'Pnw_St_glaas',
        'Pnw_St_br22t_cen',
        'Pnw_St_bUt_tA',
        'Pnw_St_aUf_SnIt',
        'Pnw_maa_me_St_laa_de',
        'Pnw_St_braa_ten',
        'Pnw_ge_St_myy_ze',
        'Pnw_kA_St_tO_feln',
        'Pnw_St_raIs',
        'Pnw_St_pO_mes',
        'Pnw_St_naac_tIS',
        'Pnw_St_pU_dIN',
        'Pnw_St_joo_guAt',
        'Pnw_St_frYc_te',
        'Pnw_St_eeAt_bee_re',
        'Pnw_St_ap_fel',
        'Pnw_St_ap_fel_St_zii_ne',
        'Pnw_St_man_da_St_rii_ne',
        'Pnw_St_rInt',
        'Pnw_St_SvaIn',
        'Pnw_St_hyyn_cen',
        'Pnw_St_mIlc',
        'Pnw_ge_St_vyA_tse',
        'Pnw_St_zalts',
        'Pnw_St_fE_fA',
        'Pnw_za_St_laat',
        'Pnw_St_pa_prii_kaa',
        'Pnw_St_guA_ke',
        'Pnw_to_St_maa_te',
        'Pnw_St_vuAst',
        'Pnw_za_St_laa_mii',
        'Pnw_St_kEE_ze',
        'Pnw_St_kee_kse',
        'Pnw_So_ko_St_laa_de',
        'Pnw_pra_St_lii_nen',
        'Pnw_St_zaa_ne',
        'Pnw_ge_St_hak_tes',
        'Pnw_St_flaIS',
        'Pnw_St_tiif_kyyl_St_kOst',
        'Pnw_lI_St_k2A',
        'Pnw_St_zEkt',
        'Pnw_St_koo_laa',
        'Pnw_St_lI_mo_St_naa_de',
        'Pnw_St_va_sA',
        'Pnw_St_pIt_tsaa',
        'Pnw_St_aIn_tOpf',
        'Pnw_St_vaIn',
        'Pnw_St_biA',
        'Pnw_St_zaft',
        'Pnw_St_ka_fee',
        'Pnw_St_tee',
       
        # Tageszeiten, ...:
        'Pnw_St_mOA_gen',
        'Pnw_St_mIt_tac',
        'Pnw_St_naac_mIt_tac',
        'Pnw_St_aa_bent',
        'Pnw_St_nact',
        'Pnw_St_moont',
        'Pnw_St_zO_ne',
        'Pnw_St_uuA',
        'Pnw_St_taak',
        'Pnw_St_vO_ce',
        'Pnw_St_moo_naat',
        'Pnw_St_jaA',
    
        # draussen: 
        'Pnw_St_vaAm',
        'Pnw_St_kalt',
        'Pnw_St_ree_gen',
        'Pnw_St_hI_tse',
        'Pnw_St_SIAm',
        'Pnw_St_Suu_e',
        'Pnw_St_nE_se',
        'Pnw_St_byA_gA_St_StaIc',
        'Pnw_St_I_nen_St_Stat',
        'Pnw_St_valt',
        'Pnw_St_paAk',
        'Pnw_St_baNk',
        'Pnw_St_raa_zen',
        'Pnw_St_vii_ze',
        'Pnw_St_baUm',
        'Pnw_St_haUs',
        'Pnw_St_fluuk_haa_fen',
        'Pnw_St_baan_hoof',
        'Pnw_St_bUs',
        'Pnw_St_hal_te_St_StE_le',
    
        # drinnen: 
        'Pnw_St_tYA',
        'Pnw_St_lIct',
        'Pnw_St_tIS',
        'Pnw_St_voon_St_tsI_mA',
        'Pnw_St_Es_St_tsI_mA',
        'Pnw_St_Slaaf_St_tsI_mA',
        'Pnw_St_fEAn_zee_A',
        'Pnw_kOm_St_pjuu_tA',
        'Pnw_St_SraNk',
        'Pnw_ree_St_gaal',
        'Pnw_St_kuu_lii',
        'Pnw_St_SraIp_blOk',
        'Pnw_St_briif_pa_St_piA',
        'Pnw_St_briif_Um_Slaak',
        'Pnw_St_bUnt_StIft',
        'Pnw_St_blaI_StIft',
        'Pnw_St_SraIp_hEft',
        'Pnw_St_glyy_St_bIA_ne',
    
        # mit Menschen:     
        'Pnw_St_zoon',
        'Pnw_St_tOc_tA',
        'Pnw_St_paAt_nA',
        'Pnw_St_mIt_be_St_voo_nA',
        'Pnw_St_mUnt',
        'Pnw_St_aU_ge',
        'Pnw_St_haUt',
        'Pnw_ge_St_zIct',
        'Pnw_St_haa_re',
        'Pnw_St_baAt',
        'Pnw_St_aAm',
        'Pnw_St_baIn',
        'Pnw_St_fuus',
    
        # Aktivitäten: 
        'Pnw_St_ree_den',
        'Pnw_St_briif',
        'Pnw_St_Spii_len',
        'Pnw_St_kO_cen',
        'Pnw_St_aUs_St_gee_en',
        'Pnw_St_hUnt',
        'Pnw_St_kat_tse',
        'Pnw_St_fY_tAn',
        'Pnw_St_StraI_celn',
        'Pnw_St_kUs',
    
        # Zahlen-Wörter/Wortteile: 
        'Pnw_St_nUl',
        'Pnw_St_aInts',
        'Pnw_St_tsvaI',
        'Pnw_St_draI',
        'Pnw_St_fiA',
        'Pnw_St_fYmf',
        'Pnw_St_zEks',
        'Pnw_St_zii_ben',
        'Pnw_St_act',
        'Pnw_St_nOIn',
        'Pnw_St_tseen',
        'Pnw_St_Elf',
        'Pnw_St_tsv9lf',
        'Pnw_St_zsvan_tsIc',
        'Pnw_tsIc',
        'Pnw_St_hUn_dAt',
        'Pnw_St_taU_zent',
    
        # Funktionswörter: 5 
        'Pnw_St_Unt',
        'Pnw_St_oo_dA',
        'Pnw_St_dEA',
        'Pnw_St_dii',
        'Pnw_St_das',
    
        # weiterer Konsum: 6
        'Pnw_St_plEI_stEI_Sen',
        'Pnw_St_pU_pe',
        'Pnw_St_bluu_men',
        'Pnw_ge_St_SEnk_pa_St_piA',
        'Pnw_St_bInt_St_faa_den',
        'Pnw_St_klee_be_St_StraI_fen',

    ] 


In [10]:
phono_keys = [  # one-dim list!!! for words, not syllables
        # Oberbegriffe UND Wörter:
        'Pw_St_m22_bel', 
        'Pw_St_tsIm_mA',
        'Pw_St_gee_gen_Stant',
        'Pw_St_klaI_duN',
        'Pw_St_maal_tsaIt',
        'Pw_St_tEE_tIc_kaIt',
        'Pw_St_tiA',
        'Pw_St_E_sen',
        'Pw_St_trIN_ken',
        'Pw_St_mEnS',
        'Pw_St_lee_be_St_vee_zen',
        'Pw_St_k9A_pA_St_taIl',
        'Pw_St_tsaIt',
        'Pw_St_Slaa_fen',
        'Pw_St_lee_zen',
        'Pw_St_hI_mel',
        'Pw_St_vE_tA',
        'Pw_ge_St_SENk',
        'Pw_Spa_St_tsiA_gaN',
        'Pw_St_raI_ze',
        'Pw_na_St_tuA',
        'Pw_St_tsaal',
        'Pw_St_SraI_ben',
        'Pw_St_zuu_pA_St_maAkt',
        'Pw_bE_ke_St_raI',
        'Pw_St_In_tA_nEt',
        'Pw_St_m22_bel_haUs',
        'Pw_St_vaa_ren_haUs',
        'Pw_a_po_St_tee_ke',
        
        # Aufstehen / Zubettgehen: 
        'Pw_St_bEt',
        'Pw_St_dE_ke',
        'Pw_St_vE_kA',
        'Pw_St_baat',
        'Pw_St_zaI_fe',
        'Pw_St_han_tuuc',
        'Pw_St_dU_Se',
        'Pw_St_baa_de_St_wan_ne',
        'Pw_St_tsaan_pas_taa',
        'Pw_St_tsaan_bYAs_te',
        'Pw_St_bE_cA',
        'Pw_St_dee_oo',
        'Pw_St_Stuul',
        'Pw_St_klaI_dA_St_SraNk',
        'Pw_St_byy_gel',
        'Pw_St_hoo_ze',
        'Pw_St_hEmt',
        'Pw_St_Un_tA_St_hoo_ze',
        'Pw_St_slIp',
        'Pw_St_Un_tA_St_hEmt',
        'Pw_St_rOk',
        'Pw_St_bluu_ze',
        'Pw_St_trEE_nINs_St_an_tsuuk',
        'Pw_St_nact_hEmt',
        'Pw_St_buuc',
        'Pw_St_tsaIt_SrIft',
        'Pw_St_lam_pe',
        'Pw_St_Sal_tA',
        'Pw_St_hEn_dii',
        'Pw_ta_St_blE_ten',
    
         # Mahlzeiten: 
        'Pw_St_fryy_StYk',
        'Pw_St_mIt_taak_St_Es_sen',
        'Pw_St_aa_bent_St_Es_sen',
        'Pw_St_tE_lA',
        'Pw_St_ta_se',
        'Pw_St_mE_sA',
        'Pw_St_gaa_bel',
        'Pw_St_l9_fel',
        'Pw_St_glaas',
        'Pw_St_br22t_cen',
        'Pw_St_bUt_tA',
        'Pw_St_aUf_SnIt',
        'Pw_maa_me_St_laa_de',
        'Pw_St_braa_ten',
        'Pw_ge_St_myy_ze',
        'Pw_kA_St_tO_feln',
        'Pw_St_raIs',
        'Pw_St_pO_mes',
        'Pw_St_naac_tIS',
        'Pw_St_pU_dIN',
        'Pw_St_joo_guAt',
        'Pw_St_frYc_te',
        'Pw_St_eeAt_bee_re',
        'Pw_St_ap_fel',
        'Pw_St_ap_fel_St_zii_ne',
        'Pw_St_man_da_St_rii_ne',
        'Pw_St_rInt',
        'Pw_St_SvaIn',
        'Pw_St_hyyn_cen',
        'Pw_St_mIlc',
        'Pw_ge_St_vyA_tse',
        'Pw_St_zalts',
        'Pw_St_fE_fA',
        'Pw_za_St_laat',
        'Pw_St_pa_prii_kaa',
        'Pw_St_guA_ke',
        'Pw_to_St_maa_te',
        'Pw_St_vuAst',
        'Pw_za_St_laa_mii',
        'Pw_St_kEE_ze',
        'Pw_St_kee_kse',
        'Pw_So_ko_St_laa_de',
        'Pw_pra_St_lii_nen',
        'Pw_St_zaa_ne',
        'Pw_ge_St_hak_tes',
        'Pw_St_flaIS',
        'Pw_St_tiif_kyyl_St_kOst',
        'Pw_lI_St_k2A',
        'Pw_St_zEkt',
        'Pw_St_koo_laa',
        'Pw_St_lI_mo_St_naa_de',
        'Pw_St_va_sA',
        'Pw_St_pIt_tsaa',
        'Pw_St_aIn_tOpf',
        'Pw_St_vaIn',
        'Pw_St_biA',
        'Pw_St_zaft',
        'Pw_St_ka_fee',
        'Pw_St_tee',
       
        # Tageszeiten, ...:
        'Pw_St_mOA_gen',
        'Pw_St_mIt_tac',
        'Pw_St_naac_mIt_tac',
        'Pw_St_aa_bent',
        'Pw_St_nact',
        'Pw_St_moont',
        'Pw_St_zO_ne',
        'Pw_St_uuA',
        'Pw_St_taak',
        'Pw_St_vO_ce',
        'Pw_St_moo_naat',
        'Pw_St_jaA',
    
        # draussen: 
        'Pw_St_vaAm',
        'Pw_St_kalt',
        'Pw_St_ree_gen',
        'Pw_St_hI_tse',
        'Pw_St_SIAm',
        'Pw_St_Suu_e',
        'Pw_St_nE_se',
        'Pw_St_byA_gA_St_StaIc',
        'Pw_St_I_nen_St_Stat',
        'Pw_St_valt',
        'Pw_St_paAk',
        'Pw_St_baNk',
        'Pw_St_raa_zen',
        'Pw_St_vii_ze',
        'Pw_St_baUm',
        'Pw_St_haUs',
        'Pw_St_fluuk_haa_fen',
        'Pw_St_baan_hoof',
        'Pw_St_bUs',
        'Pw_St_hal_te_St_StE_le',
    
        # drinnen: 
        'Pw_St_tYA',
        'Pw_St_lIct',
        'Pw_St_tIS',
        'Pw_St_voon_St_tsI_mA',
        'Pw_St_Es_St_tsI_mA',
        'Pw_St_Slaaf_St_tsI_mA',
        'Pw_St_fEAn_zee_A',
        'Pw_kOm_St_pjuu_tA',
        'Pw_St_SraNk',
        'Pw_ree_St_gaal',
        'Pw_St_kuu_lii',
        'Pw_St_SraIp_blOk',
        'Pw_St_briif_pa_St_piA',
        'Pw_St_briif_Um_Slaak',
        'Pw_St_bUnt_StIft',
        'Pw_St_blaI_StIft',
        'Pw_St_SraIp_hEft',
        'Pw_St_glyy_St_bIA_ne',
    
        # mit Menschen:     
        'Pw_St_zoon',
        'Pw_St_tOc_tA',
        'Pw_St_paAt_nA',
        'Pw_St_mIt_be_St_voo_nA',
        'Pw_St_mUnt',
        'Pw_St_aU_ge',
        'Pw_St_haUt',
        'Pw_ge_St_zIct',
        'Pw_St_haa_re',
        'Pw_St_baAt',
        'Pw_St_aAm',
        'Pw_St_baIn',
        'Pw_St_fuus',
    
        # Aktivitäten: 
        'Pw_St_ree_den',
        'Pw_St_briif',
        'Pw_St_Spii_len',
        'Pw_St_kO_cen',
        'Pw_St_aUs_St_gee_en',
        'Pw_St_hUnt',
        'Pw_St_kat_tse',
        'Pw_St_fY_tAn',
        'Pw_St_StraI_celn',
        'Pw_St_kUs',
    
        # Zahlen-Wörter/Wortteile: 
        'Pw_St_nUl',
        'Pw_St_aInts',
        'Pw_St_tsvaI',
        'Pw_St_draI',
        'Pw_St_fiA',
        'Pw_St_fYmf',
        'Pw_St_zEks',
        'Pw_St_zii_ben',
        'Pw_St_act',
        'Pw_St_nOIn',
        'Pw_St_tseen',
        'Pw_St_Elf',
        'Pw_St_tsv9lf',
        'Pw_St_zsvan_tsIc',
        'Pw_tsIc',
        'Pw_St_hUn_dAt',
        'Pw_St_taU_zent',
    
        # Funktionswort 
        'Pw_St_Unt',
        'Pw_St_oo_dA',
        'Pw_St_dEA',
        'Pw_St_dii',
        'Pw_St_das',
    
        # weiterer Konsum:
        'Pw_St_plEI_stEI_Sen',
        'Pw_St_pU_pe',
        'Pw_St_bluu_men',
        'Pw_ge_St_SEnk_pa_St_piA',
        'Pw_St_bInt_St_faa_den',
        'Pw_St_klee_be_St_StraI_fen',

] 



In [11]:
visual_keys = [  # one-dim list!!!
        # Oberbegriffe UND Wörter:
        'V_Moebel', 
        'V_Zimmer',
        'V_Gegenstand',
        'V_Kleidung',
        'V_Mahlzeit',
        'V_Taetigkeit',
        'V_Tier',
        'V_Essen',
        'V_Trinken',
        'V_Mensch',
        'V_Lebewesen',
        'V_Koerperteil',
        'V_Zeit',
        'V_Schlafen',
        'V_Lesen',
        'V_Himmel',
        'V_Wetter',
        'V_Geschenk',
        'V_Spaziergang',
        'V_Reise',
        'V_Natur',
        'V_Zahl',
        'V_Schreiben',
        'V_Supermarkt',
        'V_Baeckerei',
        'V_Internet',
        'V_Moebelhaus',
        'V_Warenhaus',
        'V_Apotheke',

        # Aufstehen / Zubettgehen: 
        'V_Bett',
        'V_Decke',
        'V_Wecker',
        'V_Bad',
        'V_Seife',
        'V_Handtuch',
        'V_Dusche',
        'V_Badewanne',
        'V_Zahnpasta',
        'V_Zahnbuerste',
        'V_Becher',
        'V_Deo',
        'V_Stuhl',
        'V_Kleiderschrank',
        'V_Buegel',
        'V_Hose',
        'V_Hemd',
        'V_Unterhose',
        'V_Slip',
        'V_Unterhemd',
        'V_Rock',
        'V_Bluse',
        'V_Trainingsanzug',
        'V_Nachthemd',
        'V_Buch',
        'V_Zeitschrift',
        'V_Lampe',
        'V_Schalter',
        'V_Handy',
        'V_Tabletten',
    
         # Mahlzeiten: 
        'V_Fruehstueck',
        'V_Mittagessen',
        'V_Abendessen',
        'V_Teller',
        'V_Tasse',
        'V_Messer',
        'V_Gabel',
        'V_Loeffel',
        'V_Glas',
        'V_Broetchen',
        'V_Butter',
        'V_Aufschnitt',
        'V_Marmelade',
        'V_Braten',
        'V_Gemuese',
        'V_Kartoffeln',
        'V_Reis',
        'V_Pommes',
        'V_Nachtisch',
        'V_Pudding',
        'V_Jogurt',
        'V_Fruechte',
        'V_Erdbeere',
        'V_Apfel',
        'V_Apfelsine',
        'V_Mandarine',
        'V_Rind',
        'V_Schwein',
        'V_Huehnchen',
        'V_Milch',
        'V_Gewuerze',
        'V_Salz',
        'V_Pfeffer',
        'V_Salat',
        'V_Paprika',
        'V_Gurke',
        'V_Tomate',
        'V_Wurst',
        'V_Salami',
        'V_Kaese',
        'V_Kekse',
        'V_Schokolade',
        'V_Pralinen',
        'V_Sahne',
        'V_Gehacktes',
        'V_Fleisch',
        'V_Tiefkuehlkost',
        'V_Likoer',
        'V_Sekt',
        'V_Kola',
        'V_Limonade',
        'V_Wasser',
        'V_Pizza',
        'V_Eintopf',
        'V_Wein',
        'V_Bier',
        'V_Saft',
        'V_Kaffee',
        'V_Tee',
       
        # Tageszeiten, ...:
        'V_Morgen',
        'V_Mittag',
        'V_Nachmittag',
        'V_Abend',
        'V_Nacht',
        'V_Mond',
        'V_Sonne',
        'V_Uhr',
        'V_Tag',
        'V_Woche',
        'V_Monat',
        'V_Jahr',
    
        # draussen: 
        'V_Warm',
        'V_Kalt',
        'V_Regen',
        'V_Hitze',
        'V_Schirm',
        'V_Schuhe',
        'V_Naesse',
        'V_Buergersteig',
        'V_Innenstadt',
        'V_Wald',
        'V_Park',
        'V_Bank',
        'V_Rasen',
        'V_Wiese',
        'V_Baum',
        'V_Haus',
        'V_Flughafen',
        'V_Bahnhof',
        'V_Bus',
        'V_Haltestelle',
    
        # drinnen: 
        'V_Tuer',
        'V_Licht',
        'V_Tisch',
        'V_Wohnzimmer',
        'V_Esszimmer',
        'V_Schlafzimmer',
        'V_Fernseher',
        'V_Computer',
        'V_Schrank',
        'V_Regal',
        'V_Kuli',
        'V_Schreibblock',
        'V_Briefpapier',
        'V_Briefumschlag',
        'V_Buntstift',
        'V_Bleistift',
        'V_Schreibheft',
        'V_Gluehbirne',
    
        # mit Menschen:     
        'V_Sohn',
        'V_Tochter',
        'V_Partner',
        'V_Mitbewohner',
        'V_Mund',
        'V_Auge',
        'V_Haut',
        'V_Gesicht',
        'V_Haare',
        'V_Bart',
        'V_Arm',
        'V_Bein',
        'V_Fuss',
    
        # Aktivitäten: 
        'V_Reden',
        'V_Brief',
        'V_Spielen',
        'V_Kochen',
        'V_Ausgehen',
        'V_Hund',
        'V_Katze',
        'V_Fuettern',
        'V_Streicheln',
        'V_Kuss',
    
        # Zahlen-Wörter/Wortteile: 
        'V_Null',
        'V_Eins',
        'V_Zwei',
        'V_Drei',
        'V_Vier',
        'V_Fuenf',
        'V_Sechs',
        'V_Sieben',
        'V_Acht',
        'V_Neun',
        'V_Zehn',
        'V_Elf',
        'V_Zwoelf',
        'V_Zwanzig',
        'V_Zig',
        'V_Hundert',
        'V_Tausend',
    
        # Funktionswort 
        'V_Und',
        'V_Oder',
        'V_Der',
        'V_Die',
        'V_Das',
    
        # weiterer Konsum:
        'V_Playstation',
        'V_Puppe',
        'V_Blumen',
        'V_Geschenkpapier',
        'V_Bindfaden',
        'V_Klebestreifen',


] 


In [12]:
visual_in_keys = [  # one-dim list!!!
        # Oberbegriffe UND Wörter:
        'Vin_Moebel', 
        'Vin_Zimmer',
        'Vin_Gegenstand',
        'Vin_Kleidung',
        'Vin_Mahlzeit',
        'Vin_Taetigkeit',
        'Vin_Tier',
        'Vin_Essen',
        'Vin_Trinken',
        'Vin_Mensch',
        'Vin_Lebewesen',
        'Vin_Koerperteil',
        'Vin_Zeit',
        'Vin_Schlafen',
        'Vin_Lesen',
        'Vin_Himmel',
        'Vin_Wetter',
        'Vin_Geschenk',
        'Vin_Spaziergang',
        'Vin_Reise',
        'Vin_Natur',
        'Vin_Zahl',
        'Vin_Schreiben',
        'Vin_Supermarkt',
        'Vin_Baeckerei',
        'Vin_Internet',
        'Vin_Moebelhaus',
        'Vin_Warenhaus',
        'Vin_Apotheke',

        # Aufstehen / Zubettgehen: 
        'Vin_Bett',
        'Vin_Decke',
        'Vin_Wecker',
        'Vin_Bad',
        'Vin_Seife',
        'Vin_Handtuch',
        'Vin_Dusche',
        'Vin_Badewanne',
        'Vin_Zahnpasta',
        'Vin_Zahnbuerste',
        'Vin_Becher',
        'Vin_Deo',
        'Vin_Stuhl',
        'Vin_Kleiderschrank',
        'Vin_Buegel',
        'Vin_Hose',
        'Vin_Hemd',
        'Vin_Unterhose',
        'Vin_Slip',
        'Vin_Unterhemd',
        'Vin_Rock',
        'Vin_Bluse',
        'Vin_Trainingsanzug',
        'Vin_Nachthemd',
        'Vin_Buch',
        'Vin_Zeitschrift',
        'Vin_Lampe',
        'Vin_Schalter',
        'Vin_Handy',
        'Vin_Tabletten',
    
         # Mahlzeiten: 
        'Vin_Fruehstueck',
        'Vin_Mittagessen',
        'Vin_Abendessen',
        'Vin_Teller',
        'Vin_Tasse',
        'Vin_Messer',
        'Vin_Gabel',
        'Vin_Loeffel',
        'Vin_Glas',
        'Vin_Broetchen',
        'Vin_Butter',
        'Vin_Aufschnitt',
        'Vin_Marmelade',
        'Vin_Braten',
        'Vin_Gemuese',
        'Vin_Kartoffeln',
        'Vin_Reis',
        'Vin_Pommes',
        'Vin_Nachtisch',
        'Vin_Pudding',
        'Vin_Jogurt',
        'Vin_Fruechte',
        'Vin_Erdbeere',
        'Vin_Apfel',
        'Vin_Apfelsine',
        'Vin_Mandarine',
        'Vin_Rind',
        'Vin_Schwein',
        'Vin_Huehnchen',
        'Vin_Milch',
        'Vin_Gewuerze',
        'Vin_Salz',
        'Vin_Pfeffer',
        'Vin_Salat',
        'Vin_Paprika',
        'Vin_Gurke',
        'Vin_Tomate',
        'Vin_Wurst',
        'Vin_Salami',
        'Vin_Kaese',
        'Vin_Kekse',
        'Vin_Schokolade',
        'Vin_Pralinen',
        'Vin_Sahne',
        'Vin_Gehacktes',
        'Vin_Fleisch',
        'Vin_Tiefkuehlkost',
        'Vin_Likoer',
        'Vin_Sekt',
        'Vin_Kola',
        'Vin_Limonade',
        'Vin_Wasser',
        'Vin_Pizza',
        'Vin_Eintopf',
        'Vin_Wein',
        'Vin_Bier',
        'Vin_Saft',
        'Vin_Kaffee',
        'Vin_Tee',
       
        # Tageszeiten, ...:
        'Vin_Morgen',
        'Vin_Mittag',
        'Vin_Nachmittag',
        'Vin_Abend',
        'Vin_Nacht',
        'Vin_Mond',
        'Vin_Sonne',
        'Vin_Uhr',
        'Vin_Tag',
        'Vin_Woche',
        'Vin_Monat',
        'Vin_Jahr',
    
        # draussen: 
        'Vin_Warm',
        'Vin_Kalt',
        'Vin_Regen',
        'Vin_Hitze',
        'Vin_Schirm',
        'Vin_SchuheVin',
        'Vin_NaessVine',
        'Vin_BuergVinersteig',
        'Vin_InnenVinstadt',
        'Vin_WalVind',
        'Vin_ParVink',
        'Vin_BanVink',
        'Vin_RasVinen',
        'Vin_WieVinse',
        'Vin_BauVinm',
        'Vin_Haus',
        'Vin_Flughafen',
        'Vin_Bahnhof',
        'Vin_Bus',
        'Vin_Haltestelle',
    
        # drinnen: 
        'Vin_Tuer',
        'Vin_Licht',
        'Vin_Tisch',
        'Vin_Wohnzimmer',
        'Vin_Esszimmer',
        'Vin_Schlafzimmer',
        'Vin_Fernseher',
        'Vin_Computer',
        'Vin_Schrank',
        'Vin_Regal',
        'Vin_Kuli',
        'Vin_Schreibblock',
        'Vin_Briefpapier',
        'Vin_Briefumschlag',
        'Vin_Buntstift',
        'Vin_Bleistift',
        'Vin_Schreibheft',
        'Vin_Gluehbirne',
    
        # mit Menschen:     
        'Vin_Sohn',
        'Vin_Tochter',
        'Vin_Partner',
        'Vin_Mitbewohner',
        'Vin_Mund',
        'Vin_Auge',
        'Vin_Haut',
        'Vin_Gesicht',
        'Vin_Haare',
        'Vin_Bart',
        'Vin_Arm',
        'Vin_Bein',
        'Vin_Fuss',
    
        # Aktivitäten: 
        'Vin_Reden',
        'Vin_Brief',
        'Vin_Spielen',
        'Vin_Kochen',
        'Vin_Ausgehen',
        'Vin_Hund',
        'Vin_Katze',
        'Vin_Fuettern',
        'Vin_Streicheln',
        'Vin_Kuss',
    
        # Zahlen-Wörter/Wortteile: 
        'Vin_Null',
        'Vin_Eins',
        'Vin_Zwei',
        'Vin_Drei',
        'Vin_Vier',
        'Vin_Fuenf',
        'Vin_Sechs',
        'Vin_Sieben',
        'Vin_Acht',
        'Vin_Neun',
        'Vin_Zehn',
        'Vin_Elf',
        'Vin_Zwoelf',
        'Vin_Zwanzig',
        'Vin_Zig',
        'Vin_Hundert',
        'Vin_Tausend',
    
        # Funktionswort 
        'Vin_Und',
        'Vin_Oder',
        'Vin_Der',
        'Vin_Die',
        'Vin_Das',
    
        # weiterer Konsum:
        'Vin_Playstation',
        'Vin_Puppe',
        'Vin_Blumen',
        'Vin_Geschenkpapier',
        'Vin_Bindfaden',
        'Vin_Klebestreifen',

]

visual_in_keys2 = [ 'Vin_Nil']


In [13]:
audio_keys = [  # one-dim list!!!
        # Oberbegriffe UND Wörter:
        'A_Moebel', 
        'A_Zimmer',
        'A_Gegenstand',
        'A_Kleidung',
        'A_Mahlzeit',
        'A_Taetigkeit',
        'A_Tier',
        'A_Essen',
        'A_Trinken',
        'A_Mensch',
        'A_Lebewesen',
        'A_Koerperteil',
        'A_Zeit',
        'A_Schlafen',
        'A_Lesen',
        'A_Himmel',
        'A_Wetter',
        'A_Geschenk',
        'A_Spaziergang',
        'A_Reise',
        'A_Natur',
        'A_Zahl',
        'A_Schreiben',
        'A_Supermarkt',
        'A_Baeckerei',
        'A_Internet',
        'A_Moebelhaus',
        'A_Warenhaus',
        'A_Apotheke',
        
        # Aufstehen / Zubettgehen: 
        'A_Bett',
        'A_Decke',
        'A_Wecker',
        'A_Bad',
        'A_Seife',
        'A_Handtuch',
        'A_Dusche',
        'A_Badewanne',
        'A_Zahnpasta',
        'A_Zahnbuerste',
        'A_Becher',
        'A_Deo',
        'A_Stuhl',
        'A_Kleiderschrank',
        'A_Buegel',
        'A_Hose',
        'A_Hemd',
        'A_Unterhose',
        'A_Slip',
        'A_Unterhemd',
        'A_Rock',
        'A_Bluse',
        'A_Trainingsanzug',
        'A_Nachthemd',
        'A_Buch',
        'A_Zeitschrift',
        'A_Lampe',
        'A_Schalter',
        'A_Handy',
        'A_Tabletten',
    
         # Mahlzeiten: 
        'A_Fruehstueck',
        'A_Mittagessen',
        'A_Abendessen',
        'A_Teller',
        'A_Tasse',
        'A_Messer',
        'A_Gabel',
        'A_Loeffel',
        'A_Glas',
        'A_Broetchen',
        'A_Butter',
        'A_Aufschnitt',
        'A_Marmelade',
        'A_Braten',
        'A_Gemuese',
        'A_Kartoffeln',
        'A_Reis',
        'A_Pommes',
        'A_Nachtisch',
        'A_Pudding',
        'A_Jogurt',
        'A_Fruechte',
        'A_Erdbeere',
        'A_Apfel',
        'A_Apfelsine',
        'A_Mandarine',
        'A_Rind',
        'A_Schwein',
        'A_Huehnchen',
        'A_Milch',
        'A_Gewuerze',
        'A_Salz',
        'A_Pfeffer',
        'A_Salat',
        'A_Paprika',
        'A_Gurke',
        'A_Tomate',
        'A_Wurst',
        'A_Salami',
        'A_Kaese',
        'A_Kekse',
        'A_Schokolade',
        'A_Pralinen',
        'A_Sahne',
        'A_Gehacktes',
        'A_Fleisch',
        'A_Tiefkuehlkost',
        'A_Likoer',
        'A_Sekt',
        'A_Kola',
        'A_Limonade',
        'A_Wasser',
        'A_Pizza',
        'A_Eintopf',
        'A_Wein',
        'A_Bier',
        'A_Saft',
        'A_Kaffee',
        'A_Tee',
       
        # Tageszeiten, ...:
        'A_Morgen',
        'A_Mittag',
        'A_Nachmittag',
        'A_Abend',
        'A_Nacht',
        'A_Mond',
        'A_Sonne',
        'A_Uhr',
        'A_Tag',
        'A_Woche',
        'A_Monat',
        'A_Jahr',
    
        # draussen: 
        'A_Warm',
        'A_Kalt',
        'A_Regen',
        'A_Hitze',
        'A_Schirm',
        'A_Schuhe',
        'A_Naesse',
        'A_Buergersteig',
        'A_Innenstadt',
        'A_Wald',
        'A_Park',
        'A_Bank',
        'A_Rasen',
        'A_Wiese',
        'A_Baum',
        'A_Haus',
        'A_Flughafen',
        'A_Bahnhof',
        'A_Bus',
        'A_Haltestelle',
    
        # drinnen: 
        'A_Tuer',
        'A_Licht',
        'A_Tisch',
        'A_Wohnzimmer',
        'A_Esszimmer',
        'A_Schlafzimmer',
        'A_Fernseher',
        'A_Computer',
        'A_Schrank',
        'A_Regal',
        'A_Kuli',
        'A_Schreibblock',
        'A_Briefpapier',
        'A_Briefumschlag',
        'A_Buntstift',
        'A_Bleistift',
        'A_Schreibheft',
        'A_Gluehbirne',
    
        # mit Menschen:     
        'A_Sohn',
        'A_Tochter',
        'A_Partner',
        'A_Mitbewohner',
        'A_Mund',
        'A_Auge',
        'A_Haut',
        'A_Gesicht',
        'A_Haare',
        'A_Bart',
        'A_Arm',
        'A_Bein',
        'A_Fuss',
    
        # Aktivitäten: 
        'A_Reden',
        'A_Brief',
        'A_Spielen',
        'A_Kochen',
        'A_Ausgehen',
        'A_Hund',
        'A_Katze',
        'A_Fuettern',
        'A_Streicheln',
        'A_Kuss',
    
        # Zahlen-Wörter/Wortteile: 
        'A_Null',
        'A_Eins',
        'A_Zwei',
        'A_Drei',
        'A_Vier',
        'A_Fuenf',
        'A_Sechs',
        'A_Sieben',
        'A_Acht',
        'A_Neun',
        'A_Zehn',
        'A_Elf',
        'A_Zwoelf',
        'A_Zwanzig',
        'A_Zig',
        'A_Hundert',
        'A_Tausend',
    
        # Funktionswort 
        'A_Und',
        'A_Oder',
        'A_Der',
        'A_Die',
        'A_Das',
    
        # weiterer Konsum:
        'A_Playstation',
        'A_Puppe',
        'A_Blumen',
        'A_Geschenkpapier',
        'A_Bindfaden',
        'A_Klebestreifen',

]



In [14]:
audio_in_keys = [  # one-dim list!!!
        # Oberbegriffe UND Wörter:
        'Ain_Moebel', 
        'Ain_Zimmer',
        'Ain_Gegenstand',
        'Ain_Kleidung',
        'Ain_Mahlzeit',
        'Ain_Taetigkeit',
        'Ain_Tier',
        'Ain_Essen',
        'Ain_Trinken',
        'Ain_Mensch',
        'Ain_Lebewesen',
        'Ain_Koerperteil',
        'Ain_Zeit',
        'Ain_Schlafen',
        'Ain_Lesen',
        'Ain_Himmel',
        'Ain_Wetter',
        'Ain_Geschenk',
        'Ain_Spaziergang',
        'Ain_Reise',
        'Ain_Natur',
        'Ain_Zahl',
        'Ain_Schreiben',
        'Ain_Supermarkt',
        'Ain_Baeckerei',
        'Ain_Internet',
        'Ain_Moebelhaus',
        'Ain_Warenhaus',
        'Ain_Apotheke',

        # Aufstehen / Zubettgehen: 
        'Ain_Bett',
        'Ain_Decke',
        'Ain_Wecker',
        'Ain_Bad',
        'Ain_Seife',
        'Ain_Handtuch',
        'Ain_Dusche',
        'Ain_Badewanne',
        'Ain_Zahnpasta',
        'Ain_Zahnbuerste',
        'Ain_Becher',
        'Ain_Deo',
        'Ain_Stuhl',
        'Ain_Kleiderschrank',
        'Ain_Buegel',
        'Ain_Hose',
        'Ain_Hemd',
        'Ain_Unterhose',
        'Ain_Slip',
        'Ain_Unterhemd',
        'Ain_Rock',
        'Ain_Bluse',
        'Ain_Trainingsanzug',
        'Ain_Nachthemd',
        'Ain_Buch',
        'Ain_Zeitschrift',
        'Ain_Lampe',
        'Ain_Schalter',
        'Ain_Handy',
        'Ain_Tabletten',
    
         # Mahlzeiten: 
        'Ain_Fruehstueck',
        'Ain_Mittagessen',
        'Ain_Abendessen',
        'Ain_Teller',
        'Ain_Tasse',
        'Ain_Messer',
        'Ain_Gabel',
        'Ain_Loeffel',
        'Ain_Glas',
        'Ain_Broetchen',
        'Ain_Butter',
        'Ain_Aufschnitt',
        'Ain_Marmelade',
        'Ain_Braten',
        'Ain_Gemuese',
        'Ain_Kartoffeln',
        'Ain_Reis',
        'Ain_Pommes',
        'Ain_Nachtisch',
        'Ain_Pudding',
        'Ain_Jogurt',
        'Ain_Fruechte',
        'Ain_Erdbeere',
        'Ain_Apfel',
        'Ain_Apfelsine',
        'Ain_Mandarine',
        'Ain_Rind',
        'Ain_Schwein',
        'Ain_Huehnchen',
        'Ain_Milch',
        'Ain_Gewuerze',
        'Ain_Salz',
        'Ain_PfeAinffer',
        'Ain_SalAinat',
        'Ain_Paprika',
        'Ain_Gurke',
        'Ain_Tomate',
        'Ain_Wurst',
        'Ain_Salami',
        'Ain_Kaese',
        'Ain_Kekse',
        'Ain_Schokolade',
        'Ain_Pralinen',
        'Ain_Sahne',
        'Ain_Gehacktes',
        'Ain_Fleisch',
        'Ain_Tiefkuehlkost',
        'Ain_Likoer',
        'Ain_Sekt',
        'Ain_Kola',
        'Ain_Limonade',
        'Ain_Wasser',
        'Ain_Pizza',
        'Ain_Eintopf',
        'Ain_Wein',
        'Ain_Bier',
        'Ain_Saft',
        'Ain_Kaffee',
        'Ain_Tee',
       
        # Tageszeiten, ...:
        'Ain_Morgen',
        'Ain_Mittag',
        'Ain_Nachmittag',
        'Ain_Abend',
        'Ain_Nacht',
        'Ain_Mond',
        'Ain_Sonne',
        'Ain_Uhr',
        'Ain_Tag',
        'Ain_Woche',
        'Ain_Monat',
        'Ain_Jahr',
    
        # draussen: 
        'Ain_Warm',
        'Ain_Kalt',
        'Ain_Regen',
        'Ain_Hitze',
        'Ain_Schirm',
        'Ain_Schuhe',
        'Ain_Naesse',
        'Ain_Buergersteig',
        'Ain_Innenstadt',
        'Ain_Wald',
        'Ain_Park',
        'Ain_Bank',
        'Ain_Rasen',
        'Ain_Wiese',
        'Ain_Baum',
        'Ain_Haus',
        'Ain_Flughafen',
        'Ain_Bahnhof',
        'Ain_Bus',
        'Ain_Haltestelle',
    
        # drinnen: 
        'Ain_Tuer',
        'Ain_Licht',
        'Ain_Tisch',
        'Ain_Wohnzimmer',
        'Ain_Esszimmer',
        'Ain_Schlafzimmer',
        'Ain_Fernseher',
        'Ain_Computer',
        'Ain_Schrank',
        'Ain_Regal',
        'Ain_Kuli',
        'Ain_Schreibblock',
        'Ain_Briefpapier',
        'Ain_Briefumschlag',
        'Ain_Buntstift',
        'Ain_Bleistift',
        'Ain_Schreibheft',
        'Ain_Gluehbirne',
    
        # mit Menschen:     
        'Ain_Sohn',
        'Ain_Tochter',
        'Ain_Partner',
        'Ain_Mitbewohner',
        'Ain_Mund',
        'Ain_Auge',
        'Ain_Haut',
        'Ain_Gesicht',
        'Ain_Haare',
        'Ain_Bart',
        'Ain_Arm',
        'Ain_Bein',
        'Ain_Fuss',
    
        # Aktivitäten: 
        'Ain_Reden',
        'Ain_Brief',
        'Ain_Spielen',
        'Ain_Kochen',
        'Ain_Ausgehen',
        'Ain_Hund',
        'Ain_Katze',
        'Ain_Fuettern',
        'Ain_Streicheln',
        'Ain_Kuss',
    
        # Zahlen-Wörter/Wortteile: 
        'Ain_Null',
        'Ain_Eins',
        'Ain_Zwei',
        'Ain_Drei',
        'Ain_Vier',
        'Ain_Fuenf',
        'Ain_Sechs',
        'Ain_Sieben',
        'Ain_Acht',
        'Ain_Neun',
        'Ain_Zehn',
        'Ain_Elf',
        'Ain_Zwoelf',
        'Ain_Zwanzig',
        'Ain_Zig',
        'Ain_Hundert',
        'Ain_Tausend',
    
        # Funktionswort 
        'Ain_Und',
        'Ain_Oder',
        'Ain_Der',
        'Ain_Die',
        'Ain_Das',
    
        # weiterer Konsum:
        'Ain_Playstation',
        'Ain_Puppe',
        'Ain_Blumen',
        'Ain_Geschenkpapier',
        'Ain_Bindfaden',
        'Ain_Klebestreifen',

]

audio_in_keys2 = [ 'Ain_Nil',
    ]



In [15]:
# ---------------------------------------------------------------------------------------------------------------------    
# Input Data for the whole DEMTECT

### wird nicht mehr benötigt !!!! -> siehe input definition unten
        
# visually presented digits: GEN_NUMBER: 
visual_in_data_1 = [ 
         'Vin_Zwei',  
         'Vin_Null',  
         'Vin_Neun', 
    ] 

visual_in_data_2 = [ 
         'Vin_Vier',  
         'Vin_Null',  
         'Vin_Fuenf', 
         'Vin_Vier', 
    ]

# visually presented text: GEN_DIGITS
visual_in_data_3 = [ 
         'Vin_Sechs',
         'Vin_Hundert',
         'Vin_Eins',  
         'Vin_Und',
         'Vin_Acht',
         'Vin_Zig',
    ]

visual_in_data_4 = [
         'Vin_Zwei',  
         'Vin_Tausend',
         'Vin_Sieben',
         'Vin_Und',
         'Vin_Zwanzig',
    ]



# auditorily presented number-words: REVERSE
audio_in_data_2 = [ 
      'Ain_Sieben', 
      'Ain_Zwei', 
    ] 

audio_in_data_3 = [ 
      'Ain_Acht', 
      'Ain_Sechs', 
    ] 

audio_in_data_4 = [ 
      'Ain_Vier', 
      'Ain_Sieben', 
      'Ain_Neun', 
    ] 

audio_in_data_5 = [ 
      'Ain_Drei', 
      'Ain_Eins', 
      'Ain_Fuenf', 
    ] 

audio_in_data_6 = [ 
      'Ain_Fuenf', 
      'Ain_Vier', 
      'Ain_Neun', 
      'Ain_Sechs', 
    ] 

audio_in_data_7 = [ 
      'Ain_Eins', 
      'Ain_Neun', 
      'Ain_Sieben', 
      'Ain_Vier', 
    ] 

audio_in_data_8 = [ 
      'Ain_Zwei', 
      'Ain_Sieben', 
      'Ain_Fuenf', 
      'Ain_Drei', 
      'Ain_Sechs', 
    ] 

audio_in_data_9 = [ 
      'Ain_Eins', 
      'Ain_Drei', 
      'Ain_Fuenf', 
      'Ain_Vier', 
      'Ain_Acht', 
    ] 

audio_in_data_10 = [ 
      'Ain_Acht', 
      'Ain_Eins', 
      'Ain_Drei', 
      'Ain_Fuenf', 
      'Ain_Vier', 
      'Ain_Zwei', 
    ] 

audio_in_data_11 = [ 
      'Ain_Vier', 
      'Ain_Eins', 
      'Ain_Zwei', 
      'Ain_Sieben', 
      'Ain_Neun', 
      'Ain_Fuenf', 
    ] 




In [16]:
control_keys = [
        'NEUTRAL', 
        'KEEP',
 
        'REPEAT',    # Recall 1, 2, 3 
        'GEN_NUMBER',  # inversion digit -> number 
        'GEN_DIGITS',  # inversion text -> digits 
        'ASSOCIATE',   # supermarkt
        'REVERSE',    # digits in reverse order 
   
        'REP_LISTEN_I01',
        'REP_LISTEN_I02',
        'REP_LISTEN_I03',
        'REP_LISTEN_I04',
        'REP_LISTEN_I05',
        'REP_LISTEN_I06',
        'REP_LISTEN_I07',
        'REP_LISTEN_I08',
        'REP_LISTEN_I09',
        'REP_LISTEN_I10',
        'REP_PRODUCE_I01',
        'REP_PRODUCE_I02',
        'REP_PRODUCE_I03',
        'REP_PRODUCE_I04',
        'REP_PRODUCE_I05',
        'REP_PRODUCE_I06',
        'REP_PRODUCE_I07',
        'REP_PRODUCE_I08',
        'REP_PRODUCE_I09',
        'REP_PRODUCE_I10',

        'REV_LISTEN_I01',
        'REV_LISTEN_I02',
        'REV_LISTEN_I03',
        'REV_LISTEN_I04',
        'REV_LISTEN_I05',
        'REV_LISTEN_I06',
        'REV_PRODUCE_I01',
        'REV_PRODUCE_I02',
        'REV_PRODUCE_I03',
        'REV_PRODUCE_I04',
        'REV_PRODUCE_I05',
        'REV_PRODUCE_I06',
]

# keys for positions of words in a word list:
position_keys = [  # one-dim list!!!
         'Pos_01',  
         'Pos_02',  
         'Pos_03',  
         'Pos_04',  
         'Pos_05',  
         'Pos_06',  
         'Pos_07',  
         'Pos_08',  
         'Pos_09',  
         'Pos_10',  
] 



In [17]:
# --------------------------------------------------------------------------------------------------------------------------
# setup semantic network
# 1st order network (learned): 
semantic_network2 = {}
semantic_network2.update(phonos_deep)
semantic_network2.update(phonos)

semantic_network1 = {}
semantic_network1.update(concepts_deep)
semantic_network1.update(concepts)
  
#print (semantic_network1)
#print (semantic_network2)

# ----------------------------------------------------------------------------------------------------------------
# Step 2: Create the model and provide input 

# ---------------------------------------------------------------------------------------------------------------------------
# 1st order network: 
# Encode the semantic network in vectors without ID-vectors
sp_vocab1, rt_vocab1, id_vocab1 = encode_semantic_network(semantic_network1, rel_type_concept_keys, D)   # allways separation of sp_ and id_ 
sp_vocab2, rt_vocab2, id_vocab2 = encode_semantic_network(semantic_network2, rel_type_phono_keys, D)   # allways separation of sp_ and id_ 

# Make subsets of the semantic network for use in different parts of the model as vocabs 
sp_vocab_concept = sp_vocab1.create_subset(spn_concept_keys)
id_vocab_concept = id_vocab1.create_subset(spn_concept_keys)
rt_vocab_concept = rt_vocab1.create_subset(rel_type_concept_keys)

sp_vocab_phono = sp_vocab2.create_subset(spn_phono_keys)
id_vocab_phono = id_vocab2.create_subset(spn_phono_keys)
rt_vocab_phono = rt_vocab2.create_subset(rel_type_phono_keys)
    
# -----------------------------------------------
# Vocab for words 
vocab_concept = Vocabulary(D, strict=False)  
vocab_concept_digits = Vocabulary(D, strict=False)  
vocab_word_digits = Vocabulary(D, strict=False)  
vocab_word = Vocabulary(D, strict=False)  
vocab_phono = Vocabulary(D, strict=False)  
vocab_visual = Vocabulary(D, strict=False)  
vocab_visual_in = Vocabulary(D, strict=False)  
vocab_audio = Vocabulary(D, strict=False)  
vocab_audio_in = Vocabulary(D, strict=False)  
vocab_control = Vocabulary(D, strict=False)
vocab_position = Vocabulary(D, strict=False)
vocab_con_pos = Vocabulary(D, strict=False)
vocab_con_digits_pos = Vocabulary(D, strict=False)

# vocab_concept ist ein subset (siehe unten)  
vocab_word.populate(';'.join(word_keys))
vocab_phono.populate(';'.join(phono_keys))
vocab_visual.populate(';'.join(visual_keys))
vocab_visual_in.populate(';'.join(visual_in_keys))
vocab_visual_in.populate(';'.join(visual_in_keys2))
vocab_audio.populate(';'.join(audio_keys))
vocab_audio_in.populate(';'.join(audio_in_keys))
vocab_audio_in.populate(';'.join(audio_in_keys2))
vocab_control.populate(';'.join(control_keys))
# vocab position istein supset (siehe unten) 
vocab_con_pos.populate(';'.join(concept_keys))
vocab_con_pos.populate(';'.join(position_keys))

# additional definition of S-pointers for bindings of positions with specific words (subtask: repetition)    
vocab_con_pos.populate(''' 
    P01_Teller = Pos_01 * C_Teller; 
    P02_Hund = Pos_02 * C_Hund;
    P03_Lampe = Pos_03 * C_Lampe;
    P04_Brief = Pos_04 * C_Brief;
    P05_Apfel = Pos_05 * C_Apfel;
    P06_Hose = Pos_06 * C_Hose;
    P07_Tisch = Pos_07 * C_Tisch;
    P08_Wiese = Pos_08 * C_Wiese;
    P09_Glas = Pos_09 * C_Glas;
    P10_Baum = Pos_10 * C_Baum;
    P01_Acht = Pos_01 * C_Acht; 
    P02_Eins = Pos_02 * C_Eins;
    P03_Drei = Pos_03 * C_Drei;
    P04_Fuenf = Pos_04 * C_Fuenf;
    P05_Vier = Pos_05 * C_Vier;
    P06_Zwei = Pos_06 * C_Zwei
    ''')

con_pos_conv_keys = ['P01_Teller', 
                 'P02_Hund', 
                 'P03_Lampe', 
                 'P04_Brief',
                 'P05_Apfel', 
                 'P06_Hose', 
                 'P07_Tisch', 
                 'P08_Wiese', 
                 'P09_Glas', 
                 'P10_Baum',
                 'P01_Acht', 
                 'P02_Eins', 
                 'P03_Drei',
                 'P04_Fuenf',
                 'P05_Vier',
                 'P06_Zwei',
                ]

# ab hier dürfen nur noch vocab subsets formuliert werden!
vocab_concept = vocab_con_pos.create_subset(concept_keys)
vocab_position = vocab_con_pos.create_subset(position_keys)
vocab_con_pos_conv = vocab_con_pos.create_subset(con_pos_conv_keys)

con_pos_keys = concept_keys + position_keys + con_pos_conv_keys
#print(list(vocab_con_pos.keys()))

vocab_concept_digits = vocab_concept.create_subset(concept_digits_keys)
vocab_word_digits = vocab_word.create_subset(word_digits_keys)

vocab_con_digits_pos_conv = vocab_con_pos.create_subset(concept_digits_keys+position_keys+con_pos_conv_keys)



/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=14, similarity=0.11)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=16, similarity=0.11)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=17, similarity=0.12)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=20, similarity=0.13)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic p

/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=78, similarity=0.21)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=79, similarity=0.20)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=80, similarity=0.20)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=81, similarity=0.23)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic p

/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=136, similarity=0.24)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=137, similarity=0.24)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=138, similarity=0.25)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=139, similarity=0.24)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semant

/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=183, similarity=0.25)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=184, similarity=0.25)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=185, similarity=0.24)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=186, similarity=0.23)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semant

/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=17, similarity=0.10)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=18, similarity=0.12)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=19, similarity=0.14)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=20, similarity=0.11)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic p

...

/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=62, similarity=0.19)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=63, similarity=0.21)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=65, similarity=0.20)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=71, similarity=0.21)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic p

/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=53, similarity=0.20)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=55, similarity=0.17)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=56, similarity=0.20)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=57, similarity=0.19)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic p

/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=29, similarity=0.15)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=33, similarity=0.16)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=36, similarity=0.13)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=37, similarity=0.14)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic p

/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=197, similarity=0.25)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=215, similarity=0.24)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=219, similarity=0.25)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=19, similarity=0.11)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semanti

/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=208, similarity=0.26)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=213, similarity=0.23)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=214, similarity=0.24)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=64, M=220, similarity=0.27)
  len(self._key2idx), best_sim))
/Users/bkroeger/anaconda/lib/python3.6/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semant

In [18]:
# global variables for simulation runs 

#'REPEAT',    # Repetition 1, 2, 3 
#'GEN_NUMBER',  # inversion digit -> number 
#'GEN_DIGITS',  # inversion text -> digits 
#'ASSOCIATE',   # supermarkt
#'REVERSE',    # digits in reverse order 

###################################################################################
# prompting input: 
# time in sec: 
# current task: REVERSE

t_keep = 0.00
t_rep = 2.00 # 1.10  

def control_input_rev(t):
     # keep:
     if t_keep+0.01 < t < t_keep+0.20:
      return 'KEEP'          # task 1, Part 1
     elif t_keep+0.20 < t < t_keep+0.50:
      return 'REV_LISTEN_I01'    #  action 1 in task rev 
     elif t_keep+0.50 < t < t_keep+0.80:
      return 'REV_LISTEN_I02'    #  action 2 in task rev 
     elif t_keep+0.80 < t < t_keep+1.10:
      return 'REV_LISTEN_I03'    #  action 3 in task rev 
     elif t_keep+1.10 < t < t_keep+1.40:
      return 'REV_LISTEN_I04'    #  action 4 in task rev 
     elif t_keep+1.40 < t < t_keep+1.70:
      return 'REV_LISTEN_I05'    #  action 5 in task rev 
     elif t_keep+1.70 < t < t_keep+2.00:
      return 'REV_LISTEN_I06'    #  action 6 in task rev 
        
     # repeat in reversed order:
     elif t_rep+0.00 < t < t_rep+0.20:
      return 'REVERSE'        # task 1, Part 2 
     # invert the repetitions here!!!!
     elif t_rep+0.20 < t < t_rep+0.50:
      return 'REV_PRODUCE_I06'   # action 11 in task rev 
     elif t_rep+0.50 < t < t_rep+0.80:
      return 'REV_PRODUCE_I05'   # action 12 in task rev 
     elif t_rep+0.80 < t < t_rep+1.10:
      return 'REV_PRODUCE_I04'   # action 13 in task rev 
     elif t_rep+1.10 < t < t_rep+1.40:
      return 'REV_PRODUCE_I03'   # action 14 in task rev 
     elif t_rep+1.40 < t < t_rep+1.70:
      return 'REV_PRODUCE_I02'   # action 15 in task rev 
     elif t_rep+1.70 < t < t_rep+2.00:
      return 'REV_PRODUCE_I01'   # action 16 in task rev 
     else:
      return 'NEUTRAL'

def visual_input_rev(t):
     if 7.80 < t < 8.00:
      return 'Vin_Zwei'
     elif 8.00 < t < 8.20:
      return 'Vin_Null'
     elif 8.20 < t < 8.40:
      return 'Vin_Neun'
     else:
      return 'Vin_Nil'

def audio_input_rev(t):
     if t_keep+0.10 < t < t_keep+0.40:
      return 'Ain_Acht'
     elif t_keep+0.40 < t < t_keep+0.70:
      return 'Ain_Eins'  
     elif t_keep+0.70 < t < t_keep+1.00:
      return 'Ain_Drei'  
     elif t_keep+1.00 < t < t_keep+1.30:
      return 'Ain_Fuenf'  
     elif t_keep+1.30 < t < t_keep+1.60:
      return 'Ain_Vier'  
     elif t_keep+1.60 < t < t_keep+1.90:
      return 'Ain_Zwei'  
     #elif t_keep+1.90 < t < t_keep+2.20:
     # return 'Ain_Tisch'  
     #elif t_keep+2.20 < t < t_keep+2.50:
     # return 'Ain_Wiese'  
     #elif t_keep+2.50 < t < t_keep+2.80:
     # return 'Ain_Glas'  
     #elif t_keep+2.80 < t < t_keep+3.10:
     # return 'Ain_Baum'  
     else:
      return 'Ain_Nil'


# -----------------------------------------------------------------------------------------------------------------------
# Make a model object with the SPA network
model = spa.Network(label='DemTect') 
# -------------------------------------------------------------------------------------------------------------------------
with model:

    # Specify the (cortical) SPA Buffers to be used with BG and Thal action selection
     # input level: 
     model.input_control = State(        
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_control)
     model.visual_in = State(                                                         
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_visual_in)
     model.visual_perc = State(        
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_visual)
     model.audio_in = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_audio_in)
     model.audio_perc = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_audio)

     # production:  
     model.concept_prod = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=sp_vocab_concept) 
     model.word_prod = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_word)
     model.phono_prod = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=sp_vocab_phono)
     model.motor_prod = State(feedback=1.0, feedback_synapse=0.1,         # Memory: articulatory buffer  
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_phono)
     # perception: 
     model.concept_perc = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=sp_vocab_concept) 
     model.word_perc = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_word)
     model.phono_perc = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=sp_vocab_phono)

     # cognitive processing:  
     model.concept_in = State( # feedback=1.0, feedback_synapse=0.1,           # no Memory
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_concept) 
     model.concept_out = State(# feedback=1.0, feedback_synapse=0.1,          # no Memory   
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_concept) 
        
     # associative memories for transformations
     # visual perception: from visual_in to visual_perc to concept_vis 
     visual_in2visual = create_mapping(visual_in_keys, visual_keys)
     model.assoc_mem_visual_in2visual_perc = ThresholdingAssocMem(
            input_vocab=vocab_visual_in, output_vocab=vocab_visual, 
            mapping=visual_in2visual, threshold=0.3, function=lambda x: x > 0.)   # 0.3 
     visual2spn_concept = create_mapping(visual_keys, spn_concept_keys)
     model.assoc_mem_visual_perc2concept_perc = ThresholdingAssocMem(
            input_vocab=vocab_visual, output_vocab=sp_vocab_concept, 
            mapping=visual2spn_concept, threshold=0.3, function=lambda x: x > 0.)   # 0.3
     
     # production: from concept_prod to phono_prod to motor_prod
     spn_concept2word = create_mapping(spn_concept_keys, word_keys)    
     model.assoc_mem_concept_prod2word_prod = ThresholdingAssocMem(
            input_vocab=sp_vocab_concept, output_vocab=vocab_word, 
            mapping = spn_concept2word, threshold=0.6, function=lambda x: x > 0.)   # 0.6 but no clean up 
     word2spn_phono = create_mapping(word_keys, spn_phono_keys)   
     model.assoc_mem_word_prod2phono_prod = ThresholdingAssocMem(
            input_vocab=vocab_word, output_vocab=sp_vocab_phono, 
            mapping=word2spn_phono, threshold=0.6, function=lambda x: x > 0.)   # 0.6 but no clean up    
     spn_phono2motor = create_mapping(spn_phono_keys, phono_keys)    
     model.assoc_mem_phono_prod2motor_prod = WTAAssocMem(
            input_vocab=sp_vocab_phono, output_vocab=vocab_phono, 
            mapping = spn_phono2motor, threshold=0.7, function=lambda x: x > 0.)   # 0.7 and clean up

     # audio perception: from audio_in to concept_perc
     audio_in2audio = create_mapping(audio_in_keys, audio_keys)
     model.assoc_mem_audio_in2audio_perc = ThresholdingAssocMem(
            input_vocab=vocab_audio_in, output_vocab=vocab_audio, 
            mapping=audio_in2audio, threshold=0.3, function=lambda x: x > 0.)   # 0.3
     audio2spn_phono = create_mapping(audio_keys, spn_phono_keys)
     model.assoc_mem_audio_perc2phono_perc = ThresholdingAssocMem(
            input_vocab=vocab_audio, output_vocab=sp_vocab_phono, 
            mapping=audio2spn_phono, threshold=0.3, function=lambda x: x > 0.)   # 0.3
     spn_phono2word = create_mapping(spn_phono_keys, word_keys)
     model.assoc_mem_phono_perc2word_perc = ThresholdingAssocMem(
            input_vocab=sp_vocab_phono, output_vocab=vocab_word, 
            mapping=spn_phono2word, threshold=0.6, function=lambda x: x > 0.)   # 0.6 but no clean up
     word2spn_concept = create_mapping(word_keys, spn_concept_keys)
     model.assoc_mem_word_perc2concept_perc = ThresholdingAssocMem(
            input_vocab=vocab_word, output_vocab=sp_vocab_concept, 
            mapping=word2spn_concept, threshold=0.6, function=lambda x: x > 0.)   # 0.6 but no clean up 
        
    # concept_perc to concept_in:
     spn_concept2concept = create_mapping(spn_concept_keys, concept_keys)
     model.assoc_mem_concept_perc2concept_in = WTAAssocMem(
            input_vocab=sp_vocab_concept, output_vocab=vocab_concept, 
            mapping=spn_concept2concept, threshold=0.7, function=lambda x: x > 0.)   # 0.7 and clean up
     # concept_out to concept_prod: 
     concept2spn_concept = create_mapping(concept_keys, spn_concept_keys)   
     model.assoc_mem_concept_out2concept_prod = WTAAssocMem(
            input_vocab=vocab_concept, output_vocab=sp_vocab_concept, 
            mapping=concept2spn_concept, threshold=0.7, function=lambda x: x > 0.)   # 0.7 and clean up
    
     # associations hier erst löschen, wenn der subtask programmiert wurde!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
     # associations: 
     # processing associations: input-buffer is concept_procAssoc_in (incl. lex. associations)
     # concept_perc to concept_procAssoc_in is done by action selection)   
     # do a inverse bindung with relation type S-Poiner BuyIn
     # put result in id-buffer proc_out and forward to concept_out    
     model.concept_procAssoc_in = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=sp_vocab_concept) 
     model.concept_procAssoc_out = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=id_vocab_concept)
     # relation type buffer 
     model.rel_type_concept = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=rt_vocab_concept)
    # binding buffer: 
     model.cconvAssoc = CircularConvolution(
            n_neurons=n_neurons, dimensions=D, invert_b=True)    
     # Connections binding: 
     nengo.Connection(model.concept_procAssoc_in.output, model.cconvAssoc.A)
     nengo.Connection(model.rel_type_concept.output, model.cconvAssoc.B)
     nengo.Connection(model.cconvAssoc.output, model.concept_procAssoc_out.input)
     # buffer and connections for: 
     # concept_procAssoc_out to concept_out buffer: 
     model.assoc_mem_concept_procAssoc_out2concept_out = WTAAssocMem(
            input_vocab=id_vocab_concept, output_vocab=vocab_concept,
            mapping=spn_concept2concept , threshold=0.3, function=lambda x: x > 0.)   #  0.3   WTA
     # procAssoc_out to concept_out  connections:  
     #model.concept_procAssoc_out.output >> model.assoc_mem_concept_procAssoc_out2concept_out.input
     #model.assoc_mem_concept_procAssoc_out2concept_out.output >> model.concept_out.input
     
     # inversion !!!!  
     # processing repetitions by inversion: input-buffer is concept_procRep_in 
     # word_perc_digits >> concept_procRep_in is done by action selection    
     # do a inverse bindung with position S-Poiners 
     # put result in normal buffer proc_out and forward to concept_out
     model.word_perc_digits = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_word_digits)
     model.word_prod_digits = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_word_digits)
     model.concept_procRev_in = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_concept_digits) 
     model.concept_procRev_out = State(
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_concept_digits)
     word_digits2concept_digits = create_mapping(word_digits_keys, concept_digits_keys)
     model.assocMem_word_perc_digits2concept_procRev_in = ThresholdingAssocMem(
            input_vocab=vocab_word_digits, output_vocab=vocab_concept_digits, 
            mapping=word_digits2concept_digits, threshold=0.6, function=lambda x: x > 0.)   # 0.6 but no clean up
     concept_digits2word_digits = create_mapping(concept_digits_keys, word_digits_keys)
     model.assocMem_concept_procRev_out2word_prod_digits = ThresholdingAssocMem(
            input_vocab=vocab_concept_digits, output_vocab=vocab_word_digits, 
            mapping=concept_digits2word_digits, threshold=0.6, function=lambda x: x > 0.)   # 0.6 but no clean up
     # connections word_perc -> word_perc_digits 
     spa.reinterpret(model.word_perc, vocab_word_digits) >> model.word_perc_digits
     # connections wprd_prod_digits -> word_prod 
     spa.reinterpret(model.word_prod_digits, vocab_word) >> model.word_prod
   
     # position buffer for repetition: 
     model.position = State(          
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_position)
     # binding for repetition: 
     model.bindRev = State(           
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_con_digits_pos_conv)
     # the cconv must be calculated in the bigger vocab vocab_con_pos; the result can be displayed in a small vocab   
     spa.reinterpret(spa.reinterpret(model.position, vocab_con_digits_pos) * 
                     spa.reinterpret(model.concept_procRev_in, vocab_con_digits_pos), vocab_con_digits_pos_conv) >> model.bindRev
     # binding for repetition: Memory 
     model.bindRevMem = State(feedback=1.0, feedback_synapse=0.3,       # Memory repetition           
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_con_digits_pos_conv)
     # add up bindRep --> bindRepMem: by action selection; fac=0.1 
     # unbinding, result to resRev:
     model.resRev = State(           
            subdimensions=subdims, neurons_per_dimension=n_neurons,
            vocab=vocab_concept_digits)
     # the cconv must be calculated in the vocab vocab_con_pos_conv; the result can be displayed in vocab_concept
     spa.reinterpret((spa.reinterpret(~model.position, vocab_con_digits_pos_conv) * 
                     model.bindRevMem), vocab_concept_digits) >> model.resRev
     # mem and conncetions for cleanup of result of unbinding
     model.assocMem_resRev2concept_procRev_out = WTAAssocMem(
            input_vocab=vocab_concept_digits, 
            mapping=concept_digits_keys, threshold=0.3, function=lambda x: x > 0.)   #  0.3   WTA
     model.resRev.output >> model.assocMem_resRev2concept_procRev_out.input
     model.assocMem_resRev2concept_procRev_out.output >> model.concept_procRev_out.input   
     # output connection: concept_procRev_out to word_prod_digits: done by action selection
        
     # Connect up the rest: 
     # production: 
     # visual_in to visual_perc: 
     model.visual_in.output >> model.assoc_mem_visual_in2visual_perc.input
     model.assoc_mem_visual_in2visual_perc.output >> model.visual_perc.input
     # visual_perc to concept_perc: 
     model.visual_perc.output >> model.assoc_mem_visual_perc2concept_perc.input
     model.assoc_mem_visual_perc2concept_perc.output >> model.concept_perc.input

     # concept_prod to word_prod:
     model.concept_prod.output >> model.assoc_mem_concept_prod2word_prod.input
     model.assoc_mem_concept_prod2word_prod.output >> model.word_prod.input
     # word_prod to phono_prod:
     model.word_prod.output >> model.assoc_mem_word_prod2phono_prod.input
     model.assoc_mem_word_prod2phono_prod.output >> model.phono_prod.input
     # phono_prod to motor_prod:
     model.phono_prod.output >> model.assoc_mem_phono_prod2motor_prod.input
     model.assoc_mem_phono_prod2motor_prod.output >> model.motor_prod.input

     # perception:
     # audio_in to audi_perc:
     model.audio_in.output >> model.assoc_mem_audio_in2audio_perc.input
     model.assoc_mem_audio_in2audio_perc.output >> model.audio_perc.input
     # audio_perc to phono_perc:
     model.audio_perc.output >> model.assoc_mem_audio_perc2phono_perc.input
     model.assoc_mem_audio_perc2phono_perc.output >> model.phono_perc.input
     # phono_perc to word_perc: 
     model.phono_perc.output >> model.assoc_mem_phono_perc2word_perc.input
     model.assoc_mem_phono_perc2word_perc.output >> model.word_perc.input
     # word_perc to concept_perc:
     model.word_perc.output >> model.assoc_mem_word_perc2concept_perc.input
     model.assoc_mem_word_perc2concept_perc.output >> model.concept_perc.input

     # concept_perc to concept_in:  
     model.concept_perc.output >> model.assoc_mem_concept_perc2concept_in.input
     model.assoc_mem_concept_perc2concept_in.output >> model.concept_in.input
     # concept_out to concept_prod:  
     model.concept_out.output >> model.assoc_mem_concept_out2concept_prod.input
     model.assoc_mem_concept_out2concept_prod.output >> model.concept_prod.input
        
     # Input:  
     spa.Transcode(control_input_rev, output_vocab=vocab_control) >> model.input_control
     spa.Transcode(visual_input_rev, output_vocab=vocab_visual_in) >> model.visual_in
     spa.Transcode(audio_input_rev, output_vocab=vocab_audio_in) >> model.audio_in

     # Grundprinzip: die gerade aktive action steht in: action_sel.thalamus.output:
     fac = 0.1
     with spa.ActionSelection() as action_sel:
        spa.ifmax("NEUTRAL", 0.3)
        # Listen and Keep for repetition: 
        spa.ifmax("KEEP", spa.dot(model.input_control, spa.sym.KEEP),
            spa.sym.KEEP >> model.input_control)  
        spa.ifmax("REV_LISTEN_I01", spa.dot(model.input_control, spa.sym.REV_LISTEN_I01),
            model.word_perc_digits.output >> model.assocMem_word_perc_digits2concept_procRev_in.input,
            model.assocMem_word_perc_digits2concept_procRev_in.output >> model.concept_procRev_in.input,
            fac*model.bindRevMem + model.bindRev >> model.bindRevMem,
            spa.sym.KEEP >> model.input_control, 
            spa.sym.Pos_01 >> model.position)  
        spa.ifmax("REV_LISTEN_I02", spa.dot(model.input_control, spa.sym.REV_LISTEN_I02),
            model.word_perc_digits.output >> model.assocMem_word_perc_digits2concept_procRev_in.input,
            model.assocMem_word_perc_digits2concept_procRev_in.output >> model.concept_procRev_in.input,      
            fac*model.bindRevMem + model.bindRev >> model.bindRevMem,
            spa.sym.KEEP >> model.input_control, 
            spa.sym.Pos_02 >> model.position)  
        spa.ifmax("REV_LISTEN_I03", spa.dot(model.input_control, spa.sym.REV_LISTEN_I03),
            model.word_perc_digits.output >> model.assocMem_word_perc_digits2concept_procRev_in.input,
            model.assocMem_word_perc_digits2concept_procRev_in.output >> model.concept_procRev_in.input,      
            fac*model.bindRevMem + model.bindRev >> model.bindRevMem,
            spa.sym.KEEP >> model.input_control, 
            spa.sym.Pos_03 >> model.position)  
        spa.ifmax("REV_LISTEN_I04", spa.dot(model.input_control, spa.sym.REV_LISTEN_I04),
            model.word_perc_digits.output >> model.assocMem_word_perc_digits2concept_procRev_in.input,
            model.assocMem_word_perc_digits2concept_procRev_in.output >> model.concept_procRev_in.input,      
            fac*model.bindRevMem + model.bindRev >> model.bindRevMem,
            spa.sym.KEEP >> model.input_control, 
            spa.sym.Pos_04 >> model.position)  
        spa.ifmax("REV_LISTEN_I05", spa.dot(model.input_control, spa.sym.REV_LISTEN_I05),
            model.word_perc_digits.output >> model.assocMem_word_perc_digits2concept_procRev_in.input,
            model.assocMem_word_perc_digits2concept_procRev_in.output >> model.concept_procRev_in.input,      
            fac*model.bindRevMem + model.bindRev >> model.bindRevMem,
            spa.sym.KEEP >> model.input_control, 
            spa.sym.Pos_05 >> model.position)  
        spa.ifmax("REV_LISTEN_I06", spa.dot(model.input_control, spa.sym.REV_LISTEN_I06),
            model.word_perc_digits.output >> model.assocMem_word_perc_digits2concept_procRev_in.input,
            model.assocMem_word_perc_digits2concept_procRev_in.output >> model.concept_procRev_in.input,      
            fac*model.bindRevMem + model.bindRev >> model.bindRevMem,
            spa.sym.KEEP >> model.input_control, 
            spa.sym.Pos_06 >> model.position)  
        # repetition in reversed order: produce the memorized items: 
        spa.ifmax("REVERSE", spa.dot(model.input_control, spa.sym.REVERSE),
            spa.sym.REPEAT >> model.input_control)  
        spa.ifmax("REV_PRODUCE_I01", spa.dot(model.input_control, spa.sym.REV_PRODUCE_I01),
            spa.sym.REPEAT >> model.input_control,
            model.concept_procRev_out.output >> model.assocMem_concept_procRev_out2word_prod_digits.input,
            model.assocMem_concept_procRev_out2word_prod_digits.output >> model.word_prod_digits.input,  
            spa.sym.Pos_01 >> model.position)  
        spa.ifmax("REV_PRODUCE_I02", spa.dot(model.input_control, spa.sym.REV_PRODUCE_I02),
            spa.sym.REPEAT >> model.input_control,  
            model.concept_procRev_out.output >> model.assocMem_concept_procRev_out2word_prod_digits.input,
            model.assocMem_concept_procRev_out2word_prod_digits.output >> model.word_prod_digits.input,  
            spa.sym.Pos_02 >> model.position)  
        spa.ifmax("REV_PRODUCE_I03", spa.dot(model.input_control, spa.sym.REV_PRODUCE_I03),
            spa.sym.REPEAT >> model.input_control,  
            model.concept_procRev_out.output >> model.assocMem_concept_procRev_out2word_prod_digits.input,
            model.assocMem_concept_procRev_out2word_prod_digits.output >> model.word_prod_digits.input,  
            spa.sym.Pos_03 >> model.position)  
        spa.ifmax("REV_PRODUCE_I04", spa.dot(model.input_control, spa.sym.REV_PRODUCE_I04),
            spa.sym.REPEAT >> model.input_control,  
            model.concept_procRev_out.output >> model.assocMem_concept_procRev_out2word_prod_digits.input,
            model.assocMem_concept_procRev_out2word_prod_digits.output >> model.word_prod_digits.input,  
            spa.sym.Pos_04 >> model.position)  
        spa.ifmax("REV_PRODUCE_I05", spa.dot(model.input_control, spa.sym.REV_PRODUCE_I05),
            spa.sym.REPEAT >> model.input_control,  
            model.concept_procRev_out.output >> model.assocMem_concept_procRev_out2word_prod_digits.input,
            model.assocMem_concept_procRev_out2word_prod_digits.output >> model.word_prod_digits.input,  
            spa.sym.Pos_05 >> model.position)  
        spa.ifmax("REV_PRODUCE_I06", spa.dot(model.input_control, spa.sym.REV_PRODUCE_I06),
            spa.sym.REPEAT >> model.input_control,  
            model.concept_procRev_out.output >> model.assocMem_concept_procRev_out2word_prod_digits.input,
            model.assocMem_concept_procRev_out2word_prod_digits.output >> model.word_prod_digits,  
            spa.sym.Pos_06 >> model.position)  
        
     # ---------------------------------------------------------------------------------------------------------------------------- 
     # probe output 
     p_selected_actions = nengo.Probe(action_sel.thalamus.output, synapse=0.01)
     p_utility = nengo.Probe(action_sel.bg.input, synapse=0.01)
     p_input_control = nengo.Probe(model.input_control.output, synapse=0.01)
     p_visual_in = nengo.Probe(model.visual_in.output, synapse=0.01)
     p_audio_in = nengo.Probe(model.audio_in.output, synapse=0.01)
     #p_concept_in = nengo.Probe(model.concept_in.output, synapse=0.01)
     p_concept_procRev_in = nengo.Probe(model.concept_procRev_in.output, synapse=0.01)
     p_position = nengo.Probe(model.position.output, synapse=0.01)
     p_bindRev = nengo.Probe(model.bindRev.output, synapse=0.01)
     p_bindRevMem = nengo.Probe(model.bindRevMem.output, synapse=0.01)
     p_resRev = nengo.Probe(model.resRev.output, synapse=0.01)
     p_rel_type_concept = nengo.Probe(model.rel_type_concept.output, synapse=0.01)
     #p_concept_out = nengo.Probe(model.concept_out.output, synapse=0.01)
     p_concept_procRev_out = nengo.Probe(model.concept_procRev_out.output, synapse=0.01)
     #p_concept_prod = nengo.Probe(model.concept_prod.output, synapse=0.01)
     p_word_prod = nengo.Probe(model.word_prod.output, synapse=0.01)
     p_phono_prod = nengo.Probe(model.phono_prod.output, synapse=0.01)
     p_motor_prod = nengo.Probe(model.motor_prod.output, synapse=0.01)
     #p_concept_perc = nengo.Probe(model.concept_perc.output, synapse=0.01)
     p_word_perc = nengo.Probe(model.word_perc.output, synapse=0.01)
     p_phono_perc = nengo.Probe(model.phono_perc.output, synapse=0.01)
     # end: with model
        
print ("model building is done")


model building is done


In [20]:
#-------------------------------
# function: 

# find name, max_similarity and t_max for n_sps most active semantic pointers for a nengo-buffer (buff-name)
# sim_data of nengo buffer need to be copied in numpy-array buff_simdata for faster processing
def most_active_SPs(n_sps, buff_simdata, vocab_keys):
 sp_len = len(vocab_keys)
 #print ('amount of SPointers = ', sp_len) 

 max_val = [-1.0] * sp_len
 name_sp = [''] * sp_len
 t_max   = [0] * sp_len
 sp_list = []
 splist_tmax = []
 splist_ymax = []
 splist_name = []   
 for sp in range(0, sp_len):
    sp_list.append(sp)
 # search max_val for all sp's and transfer name, max_val and t_max to lists 
 for sp in sp_list:
    name_sp[sp] = vocab_keys[sp]
    for t in range(0, time_msec):
        if buff_simdata[t][sp] > max_val[sp]:
            max_val[sp] = buff_simdata[t][sp]
            t_max[sp] = t
 # find n SPointer with highest similarity values 
 n = n_sps
 for i in range(0, n):
    curr_max_val = -1.0
    for sp in sp_list:
        if sp > -1:
            if max_val[sp] > curr_max_val:
                curr_max_val = max_val[sp]
                curr_t_max = t_max[sp]
                sp_max_nr = sp
    # sp with max similarity value:    
    #print (name_sp[sp_max_nr]) 
    #print ('max_similarity: ', curr_max_val, '; at time in msec: ', curr_t_max)
    splist_tmax.append(curr_t_max/1000.)
    splist_ymax.append(curr_max_val)
    splist_name.append(name_sp[sp_max_nr])   
    # change sp_list
    sp_list[sp_max_nr] = -1
 return (splist_tmax, splist_ymax, splist_name)

#------------------------------------------------------------------------

#------------------------------------------------------------------------
# run model
sim = nengo.Simulator(model, progress_bar=False)  # building ...
time_msec = 4200   # 2300       

#perc_ablate = 0.4
#for ens in model.phono_prod.all_ensembles:   # Broca
#    ablate_ensemble(ens, perc_ablate, sim)
#for ens in model.phono_perc.all_ensembles:   # Wernicke
#    ablate_ensemble(ens, perc_ablate, sim)
#for ens in model.assoc_mem_word_prod2phono_prod.all_ensembles:   # transMotor
#    ablate_ensemble(ens, perc_ablate, sim)
#for ens in model.assoc_mem_phono_perc2word_perc.all_ensembles:   # transSensory
#    ablate_ensemble(ens, perc_ablate, sim)
#for ens in model.assoc_mem_word_perc2concept_perc.all_ensembles:   # mixed
#    ablate_ensemble(ens, perc_ablate, sim)
#for ens in model.assoc_mem_concept_prod2word_prod.all_ensembles:   # mixed
#    ablate_ensemble(ens, perc_ablate, sim)
#for ens in model.assoc_mem_phono_perc2phono_prod.all_ensembles:   # conduction   + Abschalten der audio-Perception!!!
#    ablate_ensemble(ens, perc_ablate, sim)

# no more loop  
for run_nr in range (0, 1):   
  for item_nr in range(0, 1):

    # -----------------------------------------------------------------------------------------------------------------------------
    # run model 
    sim.run(time_msec/1000.)    
    
    # --------------------------------------------------------------------------------------------------------------------------
    # plot results 
    fig, axes = plt.subplots(18, sharex=True, figsize=(24, 72))

    axes[0].plot(sim.trange(), spa.similarity(sim.data[p_input_control], vocab_control))
    axes[0].legend(control_keys, fontsize='x-small', loc='upper left')
    axes[0].set_ylabel('input_control')
    axes[0].set_ylim((-0.2, 2.2))

    axes[1].plot(sim.trange(), sim.data[p_selected_actions])
    axes[1].legend(tuple(action_sel.keys()), fontsize='x-small', loc='upper left')
    axes[1].set_ylabel('action')
    axes[1].set_ylim((-0.2, 1.2))
    
    axes[2].plot(sim.trange(), sim.data[p_utility])
    axes[2].legend(tuple(action_sel.keys()), fontsize='x-small', loc='upper left')
    axes[2].set_ylabel('utility')
    axes[2].set_ylim((-0.2, 2.2))

    nmax = 12
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_visual_in], vocab_visual_in), visual_in_keys+visual_in_keys2)
    axes[3].plot(sim.trange(), spa.similarity(sim.data[p_visual_in], vocab_visual_in))
    axes[3].set_ylabel('visual_in')
    axes[3].set_ylim((-0.2, 1.2))
    for i in range(0,nmax):
        axes[3].text(splist_tmax[i], splist_ymax[i], splist_name[i])
    
    nmax = 12
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_audio_in], vocab_audio_in), audio_in_keys+audio_in_keys2)
    axes[4].plot(sim.trange(), spa.similarity(sim.data[p_audio_in], vocab_audio_in))
    axes[4].set_ylabel('audio_in')
    axes[4].set_ylim((-0.2, 1.2))
    for i in range(0,nmax):
        axes[4].text(splist_tmax[i], splist_ymax[i], splist_name[i])
        
    #nmax = 12
    #splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_concept_in], vocab_concept), concept_keys)
    #axes[5].plot(sim.trange(), spa.similarity(sim.data[p_concept_in], vocab_concept))
    #axes[5].set_ylabel('concept_in')
    #axes[5].set_ylim((-0.2, 1.2))
    #for i in range(0,nmax):
    #    axes[5].text(splist_tmax[i], splist_ymax[i], splist_name[i])

    nmax = 12
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_concept_procRev_in], vocab_concept), concept_keys)
    axes[5].plot(sim.trange(), spa.similarity(sim.data[p_concept_procRev_in], vocab_concept))
    axes[5].set_ylabel('concept_procRev_in')
    axes[5].set_ylim((-0.2, 1.2))
    for i in range(0,nmax):
        axes[5].text(splist_tmax[i], splist_ymax[i], splist_name[i])
        
    nmax = 10
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_position], vocab_position), position_keys)
    axes[6].plot(sim.trange(), spa.similarity(sim.data[p_position], vocab_position))
    axes[6].set_ylabel('position')
    axes[6].set_ylim((-0.2, 1.2))
    for i in range(0,nmax):
        axes[6].text(splist_tmax[i], splist_ymax[i], splist_name[i])

    nmax = 10
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_bindRev], vocab_con_pos_conv), con_pos_conv_keys)
    axes[7].plot(sim.trange(), spa.similarity(sim.data[p_bindRev], vocab_con_pos_conv))
    axes[7].set_ylabel('bindRev')
    axes[7].set_ylim((-0.2, 1.2))
    for i in range(0,nmax):
        axes[7].text(splist_tmax[i], splist_ymax[i], splist_name[i])
        

    nmax = 10
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_bindRevMem], vocab_con_pos_conv), con_pos_conv_keys)
    axes[8].plot(sim.trange(), spa.similarity(sim.data[p_bindRevMem], vocab_con_pos_conv))
    axes[8].set_ylabel('bindRevMem')
    axes[8].set_ylim((-0.2, 2.5))
    for i in range(0,nmax):
        axes[8].text(splist_tmax[i], splist_ymax[i], splist_name[i])
    
    nmax = 10
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_resRev], vocab_concept_digits), concept_digits_keys)
    axes[9].plot(sim.trange(), spa.similarity(sim.data[p_resRev], vocab_concept_digits))
    axes[9].set_ylabel('resRev')
    axes[9].set_ylim((-0.2, 1.2))
    for i in range(0,nmax):
        axes[9].text(splist_tmax[i], splist_ymax[i], splist_name[i])
    
    # task: association supermarket:     
    #nmax = 12
    #splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_rel_type_concept], rt_vocab_concept), rel_type_concept_keys)
    #axes[9].plot(sim.trange(), spa.similarity(sim.data[p_rel_type_concept], rt_vocab_concept))
    #axes[9].set_ylabel('rel_type_concept')
    #axes[9].set_ylim((-0.2, 1.2))
    #for i in range(0,nmax):
    #    axes[9].text(splist_tmax[i], splist_ymax[i], splist_name[i])
        
    #nmax = 12
    #splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_concept_out], vocab_concept), concept_keys)
    #axes[10].plot(sim.trange(), spa.similarity(sim.data[p_concept_out], vocab_concept))
    #axes[10].set_ylabel('concept_out')
    #axes[10].set_ylim((-0.2, 1.2))
    #for i in range(0,nmax):
    #    axes[10].text(splist_tmax[i], splist_ymax[i], splist_name[i])

    nmax = 12
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_concept_procRev_out], vocab_concept_digits), concept_digits_keys)
    axes[10].plot(sim.trange(), spa.similarity(sim.data[p_concept_procRev_out], vocab_concept_digits))
    axes[10].set_ylabel('concept_procRev_out')
    axes[10].set_ylim((-0.2, 1.2))
    for i in range(0,nmax):
        axes[10].text(splist_tmax[i], splist_ymax[i], splist_name[i])

        
    #nmax = 12
    #splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_concept_prod], sp_vocab_concept), spn_concept_keys)
    #axes[11].plot(sim.trange(), spa.similarity(sim.data[p_concept_prod], sp_vocab_concept))
    #axes[11].set_ylabel('concept_prod')
    #axes[11].set_ylim((-0.2, 1.2))
    #for i in range(0,nmax):
    #    axes[11].text(splist_tmax[i], splist_ymax[i], splist_name[i])
        
    #nmax = 12
    #splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_concept_perc], sp_vocab_concept), spn_concept_keys)
    #axes[12].plot(sim.trange(), spa.similarity(sim.data[p_concept_perc], sp_vocab_concept))
    #axes[12].set_ylabel('concept_perc')
    #axes[12].set_ylim((-0.2, 1.2))
    #for i in range(0,nmax):
    #    axes[12].text(splist_tmax[i], splist_ymax[i], splist_name[i])

    nmax = 12
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_word_prod], vocab_word), word_keys)
    axes[13].plot(sim.trange(), spa.similarity(sim.data[p_word_prod], vocab_word))
    axes[13].set_ylabel('word_prod')
    axes[13].set_ylim((-0.2, 1.2))
    for i in range(0,nmax):
        axes[13].text(splist_tmax[i], splist_ymax[i], splist_name[i])

    nmax = 12
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_word_perc], vocab_word), word_keys)
    axes[14].plot(sim.trange(), spa.similarity(sim.data[p_word_perc], vocab_word))
    axes[14].set_ylabel('word_perc')
    axes[14].set_ylim((-0.2, 1.2))
    for i in range(0,nmax):
        axes[14].text(splist_tmax[i], splist_ymax[i], splist_name[i])

    nmax = 12
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_phono_prod], sp_vocab_phono), spn_phono_keys)
    axes[15].plot(sim.trange(), spa.similarity(sim.data[p_phono_prod], sp_vocab_phono))
    axes[15].set_ylabel('phono_prod')
    axes[15].set_ylim((-0.2, 1.8))
    for i in range(0,nmax):
        axes[15].text(splist_tmax[i], splist_ymax[i], splist_name[i])

    nmax = 12
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_phono_perc], sp_vocab_phono), spn_phono_keys)
    axes[16].plot(sim.trange(), spa.similarity(sim.data[p_phono_perc], sp_vocab_phono))
    axes[16].set_ylabel('phono_perc')
    axes[16].set_ylim((-0.2, 1.2))
    for i in range(0,nmax):
        axes[16].text(splist_tmax[i], splist_ymax[i], splist_name[i])
    
    nmax = 12
    splist_tmax, splist_ymax, splist_name = most_active_SPs(nmax, spa.similarity(sim.data[p_motor_prod], vocab_phono), phono_keys)
    axes[17].plot(sim.trange(), spa.similarity(sim.data[p_motor_prod], vocab_phono))
    axes[17].set_ylabel('motor_prod')
    axes[17].set_ylim((-0.2, 2.8))
    for i in range(0,nmax):
        axes[17].text(splist_tmax[i], splist_ymax[i], splist_name[i])

    # save current result as plot
    fig.savefig('trial_000.png')    # svg format for publications: [],: [],
    print ("simulation is done")
    # reset simulation for new input  
    sim.reset()


simulation is done
